In [ ]:
pip install haversine

In [ ]:
os.getcwd()

In [ ]:
import requests
import pandas as pd
import time
import datetime
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
from matplotlib import colormaps
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as mplPolygon
import random
from shapely.geometry import Polygon, Point
import os
import math
import numpy as np
import shapely
from scipy.ndimage import binary_dilation
import sys
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
from haversine import haversine, Unit

In [ ]:
#Set working directory to the location where you downloaded the folder.
os.chdir('Downloads/Chicago-Weather/Data/')
df2 = pd.read_csv('stations.csv')
results = pd.read_csv('results.csv')
results2 = pd.read_csv('results2.csv')
All_Illinois_stations = pd.read_csv('All_Illinois_stations.csv')
datatypes = pd.read_csv('datatypes.csv')
OHare_hourly = pd.read_csv('OHare_hourly.csv')
DeKalb = pd.read_csv('DeKalb.csv')

In [ ]:
def country(x):
  return x[-2:]

In [ ]:
df2['country'] = df2.name.apply(country)

In [ ]:
#Create state column in our dataframe of locations and limit to US locations
df2['state'] = df2.name.str.extract(pat = r'(\,.+)')
df2['state'] = df2['state'].str.strip(r'(\,)')
df2['state'] = df2['state'].str.strip('US')
df2['state'] = df2['state'].str.strip()

In [ ]:
df2 = df2[df2['country'] == 'US']

##### To display the map smoothly so that the plot doesn't split the few locations in far western Alaska and the Pacific to the right side of the plot, we convert these to be negative as well.

In [ ]:
def longitude_conversion(x):
  if x > 0:
    return x - 360
  else: return x

In [ ]:
df2.longitude = df2.longitude.apply(longitude_conversion)

In [ ]:
os.chdir('../tl_2023_us_state')

In [ ]:
gdf = gpd.read_file('tl_2023_us_state.shp')

##### We convert our pandas dataframe to a geopandas data frame so we can easily plot it.

In [ ]:
gdf.geometry[13]

##### Plotting the country looks very warped due to the curvature of the planet; use the clip function to set the corners of the plot

In [ ]:
gdf.clip(Polygon(((-200, 0), (-60, 0), (-60, 75), (-200,75)))).plot()

In [ ]:
visframe = gdf.to_crs('epsg:2163')

##### Change the coordinate system to one that is better suited to plotting the US

In [ ]:
fig,ax = plt.subplots(figsize = (10,10))
visframe[~visframe['STUSPS'].isin(['HI', 'AK', 'GU', 'PR', 'AS', 'MP', 'VI'])].plot(linewidth = 0.5, **{'edgecolor': 'red', 'facecolor': 'white'}, ax = ax)

In [ ]:
unique_chi_stations = results.station.unique()
unique_chi_stations = pd.Series(unique_chi_stations)

In [ ]:
stations = list(df2.id)

In [ ]:
results['date2'] = pd.to_datetime(results.date)
results2['date2'] = pd.to_datetime(results2.date)

In [ ]:
results_2004 = results.query('date2.dt.year == 2004')

In [ ]:
os.chdir('../cityboundary/')
gdf2 = gpd.read_file('cityboundary.shp')
gdf2 = gdf2.to_crs(crs = 4326)

In [ ]:
gdf2.plot(**{'facecolor':'white', 'edgecolor':'blue'})

In [ ]:
Chi_locations = df2[df2.id.isin(unique_chi_stations)]
Chi_locations = gpd.GeoDataFrame(Chi_locations, geometry = gpd.points_from_xy(Chi_locations.longitude, Chi_locations.latitude))
Chi_locations = Chi_locations.set_index(pd.Series(range(len(Chi_locations))))

In [ ]:
Temp_df = results_2004.set_index(['date2', 'station']).query('datatype.isin(("TMAX","TMIN", "TOBS"))').pivot(columns = 'datatype', values = 'value').reset_index().merge(Chi_locations, how = "left", left_on = "station", right_on = "id")
Jan1 = gpd.GeoDataFrame(Temp_df.query('date2 == datetime.date(2004,1,1)'))

In [ ]:
ax = Jan1.plot('TOBS', cmap = 'bwr')
Jan1.apply(lambda x: ax.annotate(text = x['TOBS'], xy = x.geometry.coords[0]), axis = 1)

##### Our first attempt to visualize this across the entire city will be to create a fine two-dimensional meshgrid of points, and simply color a zone defined by all points closest to a particular station with the same color as the measurement for that station.

In [ ]:
#df2.query('id.str.contains("ACL")')

In [ ]:
Point(Jan1.geometry[0].coords[0]).distance(Point(Jan1.geometry[2].coords[0]))

##### We will try a 50x50 meshgrid of evenly spaced points from min to max longitude and latitude at first

In [ ]:
b, c = np.meshgrid(np.linspace(Jan1.longitude.min(), Jan1.longitude.max(), 50), np.linspace(Jan1.latitude.min(), Jan1.latitude.max(), 50))

In [ ]:
print((b[0,0], c[0,0]))
print(Jan1.geometry[5].coords[0])

##### Count of observations for each data type by station

In [ ]:
results.query('datatype.isin(("TMAX", "TMIN", "TOBS"))').set_index(['date2', 'station']).pivot(columns ='datatype', values = 'value').reset_index().merge(Chi_locations, how = "left", left_on = 'station', right_on = 'id').groupby(['station']).count()

##### As a test, create a list of the distance from the first station to each other station

In [ ]:
a = []
for i in range(len(Jan1)):
  a.append(haversine(Jan1.geometry[0].coords[0], Jan1.geometry[i].coords[0], unit = Unit.MILES))

In [ ]:
a = np.array(a)
a

In [ ]:
a = []
for i in range(1):
  for j in range(3):
    d = []
    for k in range(len(Jan1)):
      d.append(haversine((b[i,j], c[i,j]), (Jan1.geometry[k].coords[0])))
    a.append(np.min(np.array(d)))

##### Create an array of the closest station to each point in our mesh grid:

In [ ]:
e = []
for i in range(len(b)):
  for j in range(len(c)):
    d = []
    for k in range(len(Jan1)):
      d.append(haversine((c[i,j], b[i,j]), (Jan1.geometry[k].coords[0][::-1])))
    e.append(np.argmin(np.array(d)))

##### f is the temperature measurement at the closest station to each point in our meshgrid

In [ ]:
f = np.array(Jan1.TOBS[e])
e = np.reshape(e, (50,50))
f = np.reshape(f, (50,50))

In [ ]:
plt.contourf(b,c,e)

##### use the plt contourf function to color the zones on the 50x50 meshgrid (b and c) closest to each station. 

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
g = np.array((b,c,e))

In [ ]:
mask = np.empty((len(np.unique(e)), 50, 50))

##### use binary dilation function from scipy to define the borders of each zone. For each of the 10 stations (i), we create a mask from two conditions: the binary dilation of the second dimension of g (the numerical value of the nearest station) by kis not equal to the each i but the point on the grid is. This creates a list of the boundary points. See: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.binary_dilation.html

In [ ]:
k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
k

In [ ]:
for i in np.unique(e):
  mask[i,:,:] = binary_dilation(g[2,:,:] != i, k) & (g[2,:,:] == i)

In [ ]:
mask = mask.astype('bool')

##### Create a dictionary of data frames, one for each station, of the border points of each station

In [ ]:
border_collection = {}
for i in np.unique(e):
  border_collection[i] = pd.DataFrame({'x': g[0:2,mask[i]][0], 'y': g[0:2,mask[i]][1]})

##### Concatenate corner points to these 4 data frames so we can plot this nicely

In [ ]:
border_collection[0] = pd.concat([border_collection[0], pd.DataFrame({'x': [border_collection[0].x.min()], 'y': [border_collection[0].y.max()]})])
border_collection[1] = pd.concat([border_collection[1], pd.DataFrame({'x': [border_collection[1].x.max()], 'y': [border_collection[1].y.max()]})])
border_collection[3] = pd.concat([border_collection[3], pd.DataFrame({'x': [border_collection[3].x.max()], 'y': [border_collection[3].y.min()]})])
border_collection[4] = pd.concat([border_collection[4], pd.DataFrame({'x': [border_collection[4].x.min()], 'y': [border_collection[4].y.min()]})])

##### Reset index after the last step of concatenation so there is a unique index

In [ ]:
for i in border_collection:
  border_collection[i] = border_collection[i].reset_index()
  border_collection[i] = border_collection[i].drop('index', axis = 1)

##### Make into a geopandas dataframe

In [ ]:
for i in np.unique(e):
  border_collection[i] = gpd.GeoDataFrame(border_collection[i], geometry = gpd.points_from_xy(border_collection[i].x, border_collection[i].y))

##### The following steps will rearrange each data frame in our border_collection dictionary to have the type Multipoint from the shapely package, as a single row in a data frame of 10 rows. This will allow us to plot it easily.

In [ ]:
df = pd.DataFrame({'index':range(10)})

In [ ]:
z = []
for i in range(len(df)):
  x,y = shapely.MultiPoint(border_collection[i].geometry).convex_hull.exterior.coords.xy
  z.append(shapely.Polygon(zip(x, y)))

In [ ]:
z = pd.Series(z)
z

In [ ]:
df = df.merge(z.to_frame(), left_index = True, right_index = True).rename({0:'geometry'}, axis = 'columns')

In [ ]:
df = gpd.GeoDataFrame(df)

##### The borders look awkward with only a 50 x 50 grid. Trying again with a 500 x 500 grid minimizes the white border regions.

In [ ]:
df.plot()

In [ ]:
b, c = np.meshgrid(np.linspace(Jan1.longitude.min(), Jan1.longitude.max(), 500), np.linspace(Jan1.latitude.min(), Jan1.latitude.max(), 500))
e = []
for i in range(len(b)):
  for j in range(len(c)):
    d = []
    for k in range(len(Jan1)):
      d.append(haversine((c[i,j], b[i,j]), (Jan1.geometry[k].coords[0][::-1])))
    e.append(np.argmin(np.array(d)))
f = np.array(Jan1.TOBS[e])
e = np.reshape(e, (500,500))
f = np.reshape(f, (500,500))
g = np.array((b,c,e))
g.shape
k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1
k
mask = np.empty((len(np.unique(e)), 500, 500))
for i in np.unique(e):
  mask[i,:,:] = binary_dilation(g[2,:,:] != i, k) & (g[2,:,:] == i)
mask = mask.astype('bool')
border_collection = {}
for i in np.unique(e):
  border_collection[i] = pd.DataFrame({'x': g[0:2,mask[i]][0], 'y': g[0:2,mask[i]][1]})

border_collection[0] = pd.concat([border_collection[0], pd.DataFrame({'x': [border_collection[0].x.min()], 'y': [border_collection[0].y.max()]})])
border_collection[1] = pd.concat([border_collection[1], pd.DataFrame({'x': [border_collection[1].x.max()], 'y': [border_collection[1].y.max()]})])
border_collection[3] = pd.concat([border_collection[3], pd.DataFrame({'x': [border_collection[3].x.max()], 'y': [border_collection[3].y.min()]})])
border_collection[4] = pd.concat([border_collection[4], pd.DataFrame({'x': [border_collection[4].x.min()], 'y': [border_collection[4].y.min()]})])
for i in border_collection:
  border_collection[i] = border_collection[i].reset_index()
  border_collection[i] = border_collection[i].drop('index', axis = 1)
for i in np.unique(e):
  border_collection[i] = gpd.GeoDataFrame(border_collection[i], geometry = gpd.points_from_xy(border_collection[i].x, border_collection[i].y))
df = pd.DataFrame({'index':range(10)})
z = []
for i in range(len(df)):
  x,y = shapely.MultiPoint(border_collection[i].geometry).convex_hull.exterior.coords.xy
  z.append(shapely.Polygon(zip(x, y)))
z = pd.Series(z)
z
df = df.merge(z.to_frame(), left_index = True, right_index = True).rename({0:'geometry'}, axis = 'columns')
df = gpd.GeoDataFrame(df)

In [ ]:
df.plot()

In [ ]:
df = df.merge(Jan1['station'], left_index = True, right_index = True)

##### merge Temp_df (observations from 2004) with df and its zone borders

In [ ]:
Temp_df = Temp_df.merge(df, left_on = 'station', right_on = 'station').rename(columns = {'geometry_y': 'zone_borders'})

In [ ]:
Temp_df = Temp_df.set_geometry('zone_borders')

In [ ]:
Temp_by_date_collection = [v for k, v in Temp_df.groupby('date2')]

In [ ]:
Temp_by_date_collection[185].plot('TMIN', cmap = 'bwr', vmin = -10, vmax = 30)

In [ ]:
print(Temp_df.TOBS.min())
print(Temp_df.TOBS.max())

In [ ]:
Temp_by_date_collection[0].plot('TMIN', cmap = 'bwr', vmin = -10, vmax = 30)

##### create TMIN dataframe for TMIN observations

In [ ]:
TMIN = Temp_df[['date2', 'station', 'zone_borders', 'TMIN']].set_index(['station', 'zone_borders']).pivot(columns = 'date2', values = 'TMIN').reset_index()

In [ ]:
TMIN = TMIN.set_geometry('zone_borders')

##### pivot TMIN and so we can animate the results for each column

In [ ]:
var_names = ['Day'+ str(i+1) for i in range(361)]
TMIN_dates = TMIN.columns[2:]
TMIN.rename(columns = {old:new for old, new in zip(TMIN.columns[2:], var_names)}, inplace=True)

In [ ]:
days = TMIN.columns[2:]
date_frame = pd.DataFrame(days).rename(columns= {"date2":"day2"}).merge(pd.DataFrame(TMIN_dates), left_index = True, right_index = True)

In [ ]:
date_frame = date_frame.assign(date2 = lambda x: pd.to_datetime(x.date2).dt.date)

In [ ]:
Temp_df = Temp_df.set_geometry('zone_borders')

In [ ]:
#As a test, plot the zones on Day 1 of 2004
fig, ax = plt.subplots()
xlim = Temp_df.longitude.min(), Temp_df.longitude.max()
ylim = Temp_df.latitude.min(), Temp_df.latitude.max()
ax.set_xlim(xlim)
ax.set_ylim(ylim)
TMIN.plot(ax = ax, column = 'Day1', cmap = 'bwr', vmin = -10, vmax = 30, legend=True)
ax.annotate('Day', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')

In [ ]:
def animate(day):
    ax.clear()

    # Set the fixed axis limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Plot initial boundaries
    borders = TMIN.zone_borders.plot(ax=ax, edgecolor='blue', facecolor = 'white', linewidth=0.3)

    # Plot the data for the current year
    TMIN.plot(
         ax=ax, column = day, cmap = 'bwr', vmin = -10, vmax = 30,
    )

    ax.annotate(str(date_frame.date2[date_frame.day2 == str(day)].values[0]), xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')
animation = FuncAnimation(fig, animate, frames = days, repeat = False, interval = 300)

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
animation

##### Instead of these zones of one solid color, let's look at ways we can interpolate smoothly between stations. Let's start with griddata from scipy.interpolate

In [ ]:
test = Temp_df
test['date2'] = test.date2.astype(str)
test = test.set_geometry('geometry_x')[['latitude', 'longitude', 'date2','station', 'TMIN', 'geometry_x']].set_index(['station', 'geometry_x', 'latitude', 'longitude']).pivot(columns = 'date2', values = 'TMIN').reset_index()

In [ ]:
var_names = ['Day'+ str(i+1) for i in range(361)]
test_dates = test.columns[4:]
test.rename(columns = {old:new for old, new in zip(test.columns[4:], var_names)}, inplace=True)

##### turn our measured longitude, latitude, and temperature observations into arrays x, y, and z then use griddata to interpolate from our actual x,y points onto evenly spaced grid of xi and yi

In [ ]:
x = np.array(test['longitude'])
y = np.array(test['latitude'])
z = np.array(test['Day155'])
xi = np.linspace(Temp_df.longitude.min(), Temp_df.longitude.max(), 100)
yi = np.linspace(Temp_df.latitude.min(), Temp_df.latitude.max(), 100)
xi, yi = np.meshgrid(xi,yi)
zi = griddata((x,y), z, (xi,yi), method = 'cubic')

In [ ]:
(x,y)

##### use contourf again to plot 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.contourf(xi,yi,zi, cmap = 'bwr', vmin = -10, vmax = 30, levels = 20)
plt.colorbar()

##### we can also use tricontourf function

In [ ]:
fig, ax = plt.subplots()
cntr = ax.tricontourf(x, y, z, levels=np.linspace(-10,30,164), cmap="bwr", vmin = -10, vmax = 30)

fig.colorbar(cntr, ax=ax)
ax.plot(x, y, 'ko', ms=3)
ax.set(xlim=(x.min(), x.max()), ylim=(y.min(), y.max()))

##### need to remove NA values for some days for the triangle contours to work; we replace with the mean value

In [ ]:
test.iloc[:,5:365] = test.iloc[:,5:365].apply(lambda x: x.fillna(x.mean()), axis = 0)

In [ ]:
def animate(day):
    ax.clear()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(test.longitude, test.latitude, 'ko')
    cntr = ax.tricontourf(x, y, test[day], levels=np.linspace(-24,26,204), cmap="bwr", vmin = -10, vmax = 30)
    ax.annotate(str(date_frame.date2[date_frame.day2 == str(day)].values[0]), xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')
animation = FuncAnimation(fig, animate, frames = days, repeat = False, interval = 300)

In [ ]:
fig, ax = plt.subplots()
#ax.tricontour(x, y, z, levels=14, colors='k')
cntr = ax.tricontourf(x, y, z, levels=np.linspace(-10,30,164), cmap="bwr", vmin = -10, vmax = 30)

fig.colorbar(cntr, ax=ax)
ax.plot(x, y, 'ko', ms=3)
ax.set(xlim=(x.min(), x.max()), ylim=(y.min(), y.max()))

In [ ]:
animation

In [ ]:
a = np.zeros((2500,10))

##### We will create a custom interpolation method instead of relying on these contour functions. Again we create evenly spaced points from min to max longitude and latitude with np.linspace, then find the distance from each point on our grid to each station using the haversine function, and weight the interpolated temperature at each point by the inverse of the square of the distance. This method, known as inverse distance weighted interpolation (IDW), has no rigorous mathematical justification, but operates on the idea that things that are closer together are more alike than things farther apart. We use the inverse of the square of the distance, but the choice of exponent is arbitrary.

In [ ]:
xi = np.linspace(test.longitude.min(), test.longitude.max(), 50)
yi = np.linspace(test.latitude.min(), test.latitude.max(), 50)
xi, yi = np.meshgrid(xi,yi)
xi, yi = xi.flatten(), yi.flatten()

In [ ]:
for i in range(len(xi)):
  for j in range(len(test.geometry_x)):
    a[i,j] = haversine(test.geometry_x[j].coords[0][::-1], (yi[i], xi[i]))

In [ ]:
a = np.where(a == 0, 0.0000001, a)

In [ ]:
a_weights = 1/(a**2)

In [ ]:
b = np.zeros((2500))
for i in range(len(a_weights)):
  b[i] = a_weights[i].sum()

In [ ]:
for i in range(len(a_weights)):
  for j in range(len(test.geometry_x)):
    a_weights[i,j] /= b[i]

##### To fill NA values for missing measurements from a station on a given day, we will also weight the non-missing values at other stations by their distance from the missing station the same way

In [ ]:
actual_station_distance = np.zeros((10,10))
for i in range(len(test)):
  for j in range(len(test)):
    actual_station_distance[i,j] = haversine(test.geometry_x[i].coords[0][::-1], test.geometry_x[j].coords[0][::-1])

In [ ]:
test = Temp_df
test['date2'] = test.date2.astype(str)
test = test.set_geometry('geometry_x')[['latitude', 'longitude', 'date2','station', 'TMIN', 'geometry_x']].set_index(['station', 'geometry_x', 'latitude', 'longitude']).pivot(columns = 'date2', values = 'TMIN').reset_index()
var_names = ['Day'+ str(i+1) for i in range(361)]
test_dates = test.columns[4:]
test.rename(columns = {old:new for old, new in zip(test.columns[4:], var_names)}, inplace=True)

In [ ]:
actual_station_distance = np.where(actual_station_distance == 0, 1000000, actual_station_distance)

In [ ]:
weights = (1/(actual_station_distance**2))

In [ ]:
sum_distance = np.zeros((10))
for i in range(len(weights)):
  sum_distance[i] = weights[i].sum()

In [ ]:
for i in range(len(weights)):
  for j in range(len(weights)):
    weights[i,j] /= sum_distance[i]

In [ ]:
test.melt().query('value.isnull()')

##### replace NA values with IDW approximation and divide by the sum of non-missing weights

In [ ]:
for i in range(len(test)):
  for j in range(len(test.columns)):
    if pd.isnull(test.iloc[i,j]):
      test.iloc[i,j] = np.nansum(weights[i]*test.iloc[:,j])/np.sum(weights[i][test.index[~test.iloc[:,j].isnull()]])
    else:
      test.iloc[i,j] = test.iloc[i,j]

##### Create an array for each day with measurements and a 50x50 grid

In [ ]:
zi = np.zeros((361,50,50))

In [ ]:
for i in range(len(days)):
  zi[i,:,:] = np.dot(a_weights, test.iloc[:,i+4]).reshape(50,50)

In [ ]:
# Rough plot for a random day
fig, ax = plt.subplots()
plt.imshow(zi[95], extent=(test.longitude.min(), test.longitude.max(), test.latitude.max(), test.latitude.min()), cmap = 'bwr',  vmin = -10, vmax = 30)
ax.scatter(test.longitude, test.latitude, marker = '.', c = 'black', s = 0.1)
plt.colorbar()

##### We repeat similar steps as above to plot the 50x50 grid as an animation

In [ ]:
def plot(x,y,z,grid):
    plt.figure()
    plt.imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()), cmap = 'bwr',  vmin = -10, vmax = 30)
    plt.scatter(x,y,c=z, cmap = 'bwr', vmin = -10, vmax = 30)
    plt.colorbar()
    ax.annotate(str(date_frame.date2[i]), xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')

In [ ]:
days2 = [i for i in range(len(days))]

In [ ]:
fig = plt.figure()
ax = plt.axes()
im = ax.imshow(zi[0], extent = (test.longitude.min(), test.longitude.max(), test.latitude.min(), test.latitude.max()), cmap = 'bwr', vmin = -10, vmax = 30)
cb = fig.colorbar(im)
ax.scatter(test.longitude, test.latitude, c = 'black', s = 0.1)
ax.annotate(str(date_frame.date2[0])+f', mean temp: {test.iloc[:,4].mean().round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')

In [ ]:
def init():
  im.set_data(np.zeros((50,50)))
  return

In [ ]:
def animate(i):
  ax.clear()
  im.set_array(zi[i])
  ax.imshow(zi[i], cmap = 'bwr', extent = (test.longitude.min(), test.longitude.max(), test.latitude.min(), test.latitude.max()), vmin = -10, vmax = 30)
  ax.scatter(test.longitude, test.latitude, c = test.iloc[:, i+4], s = 0.1, cmap = 'bwr', vmin = -10, vmax = 30)
  ax.annotate(str(date_frame.date2[i])+f', mean temp: {test.iloc[:,i+4].mean().round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')

In [ ]:
animation = FuncAnimation(fig, animate, init_func = init, frames = range(len(date_frame)), interval = 300, repeat = False)
plt.show()

In [ ]:
animation

##### The animation is smooth and conveys the temperature nicely, but it lacks the spatial reference to points on the map. We want to overlay a plot of this region of Illinois on the animation.

In [ ]:
os.chdir("../tl_2023_17_tract")
gdf3 = gpd.read_file('tl_2023_17_tract.shp')

In [ ]:
clip = Polygon(((Chi_locations.longitude.min(), Chi_locations.latitude.min()), (Chi_locations.longitude.max(), Chi_locations.latitude.min()), (Chi_locations.longitude.max(), Chi_locations.latitude.max()), (Chi_locations.longitude.min(), Chi_locations.latitude.max())))
gdf4 = gdf3.clip(clip)

In [ ]:
fig, ax = plt.subplots()
gdf2.geometry.boundary.plot(ax = ax, **{'linewidth':0.5, 'edgecolor':'green'})
gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].plot(alpha = 0.5, ax = ax, **{'linewidth':0.05, 'facecolor':'turquoise'})
Chi_locations.plot(ax = ax, markersize = 0.5, **{'facecolor':'black'})
# Black dots represent station locations within the greater Chicago area; Green boundary represents the borders of the city.

##### we can superimpose a frame of our smooth temperature approximation on the map of the city, but it fits awkwardly and only covers part of the metro area.

In [ ]:
fig = plt.figure()
ax = plt.axes()
gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
im = ax.imshow(zi[0], extent = (test.longitude.min(), test.longitude.max(), test.latitude.min(), test.latitude.max()), cmap = 'bwr', vmin = -10, vmax = 30)
cb = fig.colorbar(im)
ax.scatter(test.longitude, test.latitude, c = 'black', s = 0.5)
ax.annotate(str(date_frame.date2[0])+f', mean temp: {test.iloc[:,4].mean().round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')

In [ ]:
def init():
  im.set_data(np.zeros((50,50)))
  return
def animate(i):
  ax.clear()
  im.set_array(zi[i])
  gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'green', 'facecolor':'none'})
  ax.imshow(zi[i], cmap = 'bwr', extent = (test.longitude.min(), test.longitude.max(), test.latitude.min(), test.latitude.max()), vmin = -10, vmax = 30)
  ax.scatter(test.longitude, test.latitude, c = test.iloc[:, i+4], s = 0.1, cmap = 'bwr', vmin = -10, vmax = 30)
  ax.annotate(str(date_frame.date2[i])+f', mean temp: {test.iloc[:,i+4].mean().round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')
animation = FuncAnimation(fig, animate, init_func = init, frames = range(len(date_frame)), interval = 300, repeat = False)
plt.show()

##### This animation is quite large, so we have to set a new upper limit to display it.

In [ ]:
mpl.rcParams['animation.embed_limit'] = 100

In [ ]:
animation

##### The data from 2004 is too incomplete to project temperature for the city of Chicago with confidence from just distance alone. None of the stations are locatedin the city itself, and proximity to the lake and the urban heat island effect could both significantly impact weather in the city. We expand the results to 20 years from 2004 through 2023 to see if more of the stations listed have results.

In [ ]:
results3 = pd.concat((results, results2))

In [ ]:
#Limit our 20 year results to temperature measurements
Temp_20year = results3.query('datatype.isin(("TOBS", "TMIN", "TMAX"))').drop('Unnamed: 0', axis = 1).reset_index().drop('index', axis = 1)

In [ ]:
results3.groupby('station').count().sort_values(by = 'date', ascending = False).head(20)

In [ ]:
Temp_20year['date'] = pd.to_datetime(Temp_20year.date)
Temp_20year['date2'] = pd.to_datetime(Temp_20year.date).dt.date
Temp_20year['month'] = pd.to_datetime(Temp_20year.date).dt.month
Temp_20year['dayofmonth'] = Temp_20year.date.dt.day
Temp_20year['year'] = Temp_20year.date.dt.year
Temp_20year['dayofyear'] = Temp_20year.date.dt.dayofyear

##### Many of the stations may record other values, like precipitation, but only 13 record any temperature value

In [ ]:
Temp_20year.groupby('station').count()

##### Lisle Morton Arboretum changes station id in 2007 but is otherwise identical

In [ ]:
Temp_20year.query('station.isin(("GHCND:USC00119221", "GHCND:USC00115097"))').groupby('date2').count()['station'].sort_values(ascending = False)

##### we change old values of previous Morton Arboretum station id to the new value:

In [ ]:
Temp_20year['station'] = np.where(Temp_20year.station == "GHCND:USC00115097", "GHCND:USC00119221", Temp_20year.station)

##### Starting in 2005 there is a station inside the city, near the lakeshore. We can get an idea of the effect of the lake and city.

In [ ]:
Temp_20year_stations = Temp_20year.groupby('station').count().merge(Chi_locations, how = 'left', left_on = "station", right_on = "id")
fig,ax = plt.subplots()
Temp_20year_stations = Temp_20year_stations.set_geometry(gpd.points_from_xy(Temp_20year_stations.longitude, Temp_20year_stations.latitude))
Temp_20year_stations.plot(ax = ax, marker = '.', c = 'black')
gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].plot(alpha = 0.5, ax = ax, **{'linewidth':0.05, 'facecolor':'turquoise'})

In [ ]:
Temp_20year_stations

In [ ]:
Temp_20year.query('datatype == "TMIN"')
#74662 rows

##### There's no satisfying way to adjust for a leap year and group by the ordinal day of the year across different years (January 1 of 2005 and 2006 would have different adjusted values, for instance) and the pattern would repeat every 4 years (so we could compare 2005 values to 2009). For the sake of simplicity, we will just ignore the leap year and compare day 1 of each year.

In [ ]:
#pivot data
Temp_20year = Temp_20year.set_index(['station', 'date2', 'month', 'dayofmonth', 'year', 'dayofyear']).pivot(columns = 'datatype', values = 'value').reset_index()

In [ ]:
Temp_20year

In [ ]:
Temp_20year.query('TMAX.isnull()')
#only 148 rows are missing TMIN vs 186 for TMAX and 22720 for TOBS.

In [ ]:
Temp_20year.query('TMIN.isnull() | TMAX.isnull()')['station'].count()
#305 rows

##### Explore strategies for interpolating TMIN or TMAX if missing. What is the average difference between TMIN and TMAX?

In [ ]:
Temp_20year.query('~TMIN.isnull() & ~TMAX.isnull()').assign(diff = lambda x: x.TMAX - x.TMIN).hist('diff', bins = 30)

In [ ]:
test2 = Temp_20year.query('~TMIN.isnull() & ~TMAX.isnull()').assign(diff = lambda x: x.TMAX - x.TMIN)

##### Calculate mean and standard deviation of difference between TMAX and TMIN at each station

In [ ]:
test2.groupby(['station'])['diff'].aggregate(["mean", "std"])

##### Create a histogram of temperature difference from max to min for each station to look for patterns 

In [ ]:
fig, axs = plt.subplots(4, 3)
for ax, (name, new) in zip(axs.flatten(), test2.groupby('station')):
  new.hist('diff', ax = ax, bins = 30)

##### Standard deviation is too high to simply interpolate values confidently. There are few enough missing values that we will simply ignore them. We make an average of TMIN and TMAX as a straightforward estimate of the average temperature.

In [ ]:
#create mean temperature column from average of min and max
test2 = test2.assign(TMEAN = lambda x: (x.TMIN + x.TMAX)/2).groupby(['station', 'dayofyear'])['TMEAN'].aggregate(['mean', 'std']).reset_index()

In [ ]:
test2 = test2.merge(Chi_locations, how = 'left', left_on = "station", right_on = "id")

In [ ]:
#Plot 20 year average temperature by day of year
fig, ax = plt.subplots()
ax.set_prop_cycle(color = ['blue', 'gray', 'red'])
ax.set_ylabel('Temperature in C')
Temp_20year.query('~TMIN.isnull() & ~TMAX.isnull()').assign(TMEAN = lambda x: (x.TMAX + x.TMIN)/2)\
.groupby(['month', 'dayofmonth'])[['TMIN', 'TMAX', 'TMEAN']].aggregate("mean").reset_index().reset_index()\
.plot(x = 'index', y = ['TMIN', 'TMEAN', 'TMAX'], ax = ax)
ax.set_xlabel('Day of Year')

In [ ]:
test2 = test2.sort_values(['dayofyear', 'name'])

In [ ]:
#pivot and make a column for the mean temperature at a given station on each day of the year
test2 = test2.set_index(['station', 'name', 'geometry']).pivot(columns = 'dayofyear', values = 'mean').reset_index()

In [ ]:
var_names = ['Day' +str(i+1) for i in range(366)]
test2.rename(columns = {old:new for old, new in zip(test2.columns[3:], var_names)}, inplace=True)

In [ ]:
#repeat IDW steps from above
a = np.zeros((2500,len(test2.geometry)))
xi = np.linspace(Chi_locations.longitude.min(), -87.52 , 50) #longitude of Illinois-Indiana border
yi = np.linspace(Chi_locations.latitude.min(), Chi_locations.latitude.max(), 50)
xi, yi = np.meshgrid(xi,yi)
xi, yi = xi.flatten(), yi.flatten()
for i in range(len(xi)):
  for j in range(len(test2.geometry)):
    a[i,j] = haversine(test2.geometry[j].coords[0][::-1], (yi[i], xi[i]))
a = np.where(a == 0, 0.0000001, a)
a_weights = 1/(a**2)
b = np.zeros((2500))
for i in range(len(a_weights)):
  b[i] = a_weights[i].sum()
for i in range(len(a_weights)):
  for j in range(len(test2.geometry)):
    a_weights[i,j] /= b[i]
actual_station_distance = np.zeros((12,12))
for i in range(len(test2)):
  for j in range(len(test2)):
    actual_station_distance[i,j] = haversine(test2.geometry[i].coords[0][::-1], test2.geometry[j].coords[0][::-1])
actual_station_distance = np.where(actual_station_distance == 0, 1000000, actual_station_distance)
weights = (1/(actual_station_distance**2))
sum_distance = np.zeros((len(test2)))
for i in range(len(weights)):
  sum_distance[i] = weights[i].sum()
for i in range(len(weights)):
  for j in range(len(weights)):
    weights[i,j] /= sum_distance[i]

In [ ]:
closest_station_distance = np.zeros((12))
for i in range(len(actual_station_distance)):
  closest_station_distance[i] = np.min(actual_station_distance[i])

In [ ]:
zi = np.zeros((366, 50, 50))
for i in range(366):
  zi[i,:,:] = np.dot(a_weights, test2.iloc[:,i+3]).reshape(50,50)

In [ ]:
clip = Polygon(((Chi_locations.longitude.min(), Chi_locations.latitude.min()), (-87.52, Chi_locations.latitude.min()), (-87.52, Chi_locations.latitude.max()), (Chi_locations.longitude.min(), Chi_locations.latitude.max())))

In [ ]:
fig = plt.figure()
ax = plt.axes()
gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
Temp_20year_stations.plot(ax = ax, marker = '.', c = 'black')
gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].clip(clip).plot(ax = ax, **{'linewidth':0.06, 'facecolor':'turquoise'})
im = ax.imshow(zi[0], extent = (Chi_locations.longitude.min(), -87.52, Chi_locations.latitude.min(), Chi_locations.latitude.max()), cmap = 'bwr', vmin = -10, vmax = 30)
plt.colorbar(im)

In [ ]:
def init():
  im.set_data(np.zeros((50,50)))
  return
def animate(i):
  ax.clear()
  im.set_array(zi[i])
  gdf4.clip(clip).plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
  gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].clip(clip).plot(ax = ax, **{'linewidth':0.06, 'facecolor':'turquoise'})
  ax.imshow(zi[i], cmap = 'bwr', extent = (Chi_locations.longitude.min(), -87.52, Chi_locations.latitude.min(), Chi_locations.latitude.max()), vmin = -10, vmax = 30)
  ax.scatter(Temp_20year_stations.longitude, Temp_20year_stations.latitude, marker = '.', c = "black", s = 0.1)
  ax.annotate(f' Day {str(i+1)}' f', mean temp: {test2.iloc[:,i+3].mean().round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')
anim = FuncAnimation(fig, animate, init_func = init, frames = range(365), interval = 300, repeat = False)


##### And now we have a much nice animation with the map of Chicago in the background

In [ ]:
anim

In [ ]:
#pip install ffmpeg

In [ ]:
#import ffmpeg

In [ ]:
#we save this very large animation
#writervideo = mpl.animation.FFMpegWriter(fps=60)
#anim.save('weather.mp4', writer=writervideo)

##### I'm interested in checking and visualizing how well the IDW interpolation works for known values at each station. Are some stations approximated better than others? Create an IDW approximation for each day for each station and subtract it from the measured temperature

In [ ]:
weights_copy = np.copy(weights)
#make diagonal equal to zero so that the station's own values have no effect on the IDW interpolation for that station's measurement:
for i in range(len(weights_copy)):
  weights_copy[i,i] = 0

In [ ]:
asd = Temp_20year.set_index('station').assign(TMEAN = lambda x:x.TMIN/2 + x.TMAX/2).pivot(columns = 'date2', values = 'TMEAN')

In [ ]:
asd_test = asd.reset_index().drop('station', axis = 1)

In [ ]:
missing_values_indices = [np.array(asd_test.iloc[:,j].index[asd_test.iloc[:,j].isnull()]) for j in range(len(asd_test.columns))]

In [ ]:
sum_missing_weights = np.zeros((12, 7193))

In [ ]:
for i in range(12):
  for j in range(7193):
    sum_missing_weights[i,j] = weights_copy[i][~np.isin(np.arange(12), missing_values_indices[j])].sum()

In [ ]:
IDW_values = np.zeros((12, 7193))
for i in range(len(asd)):
  for j in range(len(asd.columns)):
    IDW_values[i,j] = np.nansum(weights_copy[i] * asd.iloc[:,j])

In [ ]:
##### Adjust IDW_values by dividing by the sum of non-missing weights matrix; otherwise the weights don't sum to 1

In [ ]:
IDW_values = (IDW_values / sum_missing_weights)

In [ ]:
residual_check = pd.DataFrame(IDW_values - np.array(asd))

In [ ]:
#mean of difference betweem our IDW approximation and the measured values in Celsius
residual_check.T.mean().mean()

In [ ]:
#add station column to our matrix of errors
residual_check = asd.reset_index().station.to_frame().merge(residual_check, left_index = True, right_index = True)

In [ ]:
melt = pd.melt(residual_check, id_vars = 'station')
#add name column from 20 year stations to label histograms
melt = Temp_20year_stations[['name', 'id']].merge(melt, how = "right", left_on = 'id', right_on = 'station').drop('id', axis = 1).assign(name = lambda x: x.name.str.replace(pat = ", IL US", repl = ""))
fig, axs = plt.subplots(3,4, **{'figsize':(12,6)}, gridspec_kw={'hspace':0.4}, sharex = True)
for ax, (name, new) in zip(axs.flatten(), melt.groupby('name')):
  new.hist('value', ax = ax, bins = 30, bottom = 4)
  ax.set_title(name, fontsize = 8)



##### some histograms have very narrow spread around zero, suggesting they are very well approximated by IDW. Others have a larger spread. Labelling the histograms by the station name suggests a plausible reason for this: the two narrowest spreads are the two distinct stations at Midway Airport. When one of these stations values is missing, the weather at the other is likely to be very similar, and most of the weight in the IDW approximation will go to the other Midway Airport station.

In [ ]:
closest_station_distance = pd.Series(closest_station_distance)

In [ ]:
pd.set_option('display.max_rows', None)
closest_station_summary = Temp_20year_stations[['name', 'id']].merge(residual_check, left_on = 'id', right_on = 'station')\
.drop(['id', 'station'], axis = 1).set_index('name').aggregate(["mean", "std"], axis = 1).reset_index()\
.merge(closest_station_distance.to_frame(), left_index = True, right_index = True).rename(columns = {0: 'closest station distance'})
closest_station_summary

##### let's look at a scatter plot of closest station distance vs standard deviation to get a sense of this relationship

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0,30)
ax.set_ylim(0,3)
ax.set_xlabel('closest station distance')
ax.set_ylabel('Standard Deviation of Approximation Error')
ax.scatter(closest_station_summary['closest station distance'], closest_station_summary['std'])

##### Most of the points do follow the linear correlation we'd expect, with the exceptions of the stations Chicago Botanical Garden and Little Red School House, which have much higher error standard deviation than we'd expect based on the distance to the closest station. The Northerly Island station close to Lake Michigan is actually well approximated by IDW, considering the distance to the closest station.

In [ ]:
#let's plot again the stations labeled by name
fig, ax = plt.subplots()
gdf4.clip(clip).plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
Temp_20year_stations.plot(ax = ax, marker = '.', c = 'black')
gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].clip(clip).plot(ax = ax, **{'linewidth':0.06, 'facecolor':'turquoise'})
for i in range(len(Temp_20year_stations)):
  plt.annotate(Temp_20year_stations.name.str.replace(", IL US", "")[i], (Temp_20year_stations.longitude[i], Temp_20year_stations.latitude[i]), **{'fontsize':8})

##### Although the Botanical Garden is significantly closer to the lake than the nearest station, the lake plays little role in the measurement differences for the Little Red School House. Another reasonable explanation is that these two sites are significantly more forested than the surrounding area, making them less impacted by the Urban Heat Island effect. Of the other stations with higher error standard deviations than expected by distance, the Barrington station is in a relatively rural area, and the Palwaukee airport is heavily impacted by having the Botanical garden as its closest and most important neighbor in our approximation, while being less forested and presumably holding heat longer due to its buildings and runways. The stations that we'd expect to be most impacted by the heat island effect all have mean errors below zero (so they all have hotter temperatures on average than expected by IDW, but since these stations inside the city of Chicago are typically all close to each other, the impact is spread out). The rural Barrington station is the one whose mean approximation error is by far the furthest away from zero, and temperatures there are on average significantly cooler than we'd expect from the temperatures at its neighboring stations.

##### We can analyze pattern of residuals by week of the year (first 7 days, then 7 days therafter) to look for patterns. Doing this day by day would be too granular.

In [ ]:
Weekly_residuals = asd.T.reset_index()['date2'].to_frame().merge(residual_check.set_index('station').T.reset_index(), left_index = True, right_index = True)\
.drop('index', axis = 1).assign(weekofyear = lambda x: pd.to_datetime(x.date2).dt.day_of_year//7).drop('date2', axis = 1).groupby('weekofyear').aggregate('mean').reset_index()


In [ ]:
#Plot the average residiual difference from approximated to actual temperature by week for each station
fig = plt.figure(figsize = (12,9))
ax = plt.axes()
for i in range(len(Weekly_residuals.columns)-1):
  line, = ax.plot(Weekly_residuals.weekofyear, Weekly_residuals.iloc[:,i+1])
  line.set_label(Temp_20year_stations.name.str.replace(", IL US", "")[i])
  ax.legend(fontsize=9, ncol = 3)
ax.set_xlabel('Week')
ax.set_ylabel('Approximated Minus Actual Temperature')

##### Although it's a bit hard to see with all lines plotted together, what immediately stands out is the extreme variation in the downtown Chicago lakefront Northerly Island station, which is much cooler than expected in the summer and much warmer than expected in the winter. This is the moderating effect of the lake that we'd expect: the very large body of water takes far more time to warm up in the summer than the surrounding air, and far more time to cool in the winter.

##### Plotting each line in its own subplot makes trends clearer.

In [ ]:
fig, ax = plt.subplots(3,4, **{'figsize': (15,8)}, gridspec_kw={'hspace':0.6})
for i in range(3):
  for j in range(4):
    ax[i,j].set_ylim(-1.5,2)
    ax[i,j].plot(Weekly_residuals.weekofyear, Weekly_residuals.iloc[:,i*4 + j + 1])
    ax[i,j].set_title(Temp_20year_stations.name.str.replace(", IL US", "")[i*4 + j], fontsize = 8)
    ax[i,j].set_xlabel("Week of Year")
    ax[i,j].axhline(0, xmin = 0, xmax = 53, c = "black")

##### The 4 airport stations tend to be warmer than the surrounding area at a relatively constant rate over the year. The plot of the mean error by week at the Botanical Garden next to the trend at Northerly Island suggests that there is a moderating effect of proximity to the lake here too, though not as pronounced. Whereas the Northerly Island station is a landfill peninsula right on the lake, the Botanical Garden station is around 1.5 miles from the lakefront. Both stations are cooler than expected in the summer and warmer than expected in the winter. Conversely, the inland but forested Little Red School House station has the opposite pattern of more extreme temperatures: colder in the winter than expected and warmer summers than expected. Some of the other stations like Streamwood and Barrington follow the same pattern in a less pronounced fashion.

##### We can look at the standard deviation of the weekly residuals instead


In [ ]:
Weekly_residuals_std = asd.T.reset_index()['date2'].to_frame().merge(residual_check.set_index('station').T.reset_index(), left_index = True, right_index = True)\
.drop('index', axis = 1).assign(weekofyear = lambda x: pd.to_datetime(x.date2).dt.day_of_year//7).drop('date2', axis = 1).groupby('weekofyear').aggregate('std').reset_index()

In [ ]:
fig, ax = plt.subplots(3,4, **{'figsize': (15,8)}, gridspec_kw={'hspace':0.6})
for i in range(3):
  for j in range(4):
    ax[i,j].set_ylim(0,5)
    ax[i,j].plot(Weekly_residuals_std.weekofyear, Weekly_residuals_std.iloc[:,i*4 + j + 1])
    ax[i,j].set_title(Temp_20year_stations.name.str.replace(", IL US", "")[i*4 + j], fontsize = 8)
    ax[i,j].set_xlabel("Week of Year")

##### Almost all the stations have a noticeable drop in the standard deviation of the approximation in summer - a slight or pronounced U shape starting roughly around week 20 - with the exceptions of the two midway stations which heavily influence the predictions of each other, and the Northerly Island station, which peaks around week 20 (mid May) but doesn't go back up again in the fall. This suggests temperatures are generally more uniform in the summer - it's hot everywhere.

##### Let's compare the changes in average temperature in the Chicago metro area from the 5 year period 2004-2008 compared to the 5 year period of 2019-2023 to get a sense of how climate change is affecting the city. 

In [ ]:
Stations_All_20_years = list(Temp_20year.groupby('station')[['date2']].max().merge(Temp_20year.groupby('station')[['date2']].min(), left_index = True, right_index = True).rename({'date2_y': 'min_date','date2_x':'max_date'}, axis = 1)\
.assign(max_date = lambda x: pd.to_datetime(x.max_date), min_date = lambda x: pd.to_datetime(x.min_date)).query("max_date == '2023-12-31' & min_date == '2004-01-01'").reset_index()['station'])

In [ ]:
test3 = Temp_20year[Temp_20year.station.isin(Stations_All_20_years)].query('~TMIN.isnull() & ~TMAX.isnull() & year.between(2004,2009)').assign(diff = lambda x: x.TMAX - x.TMIN)
test3 = test3.assign(TMEAN = lambda x: (x.TMIN + x.TMAX)/2).groupby(['station', 'dayofyear'])['TMEAN'].aggregate(['mean', 'std']).reset_index()
test3 = test3.merge(Chi_locations, how = 'left', left_on = "station", right_on = "id")
test3 = test3.sort_values(['dayofyear', 'name'])
test3 = test3.set_index(['station', 'name', 'geometry']).pivot(columns = 'dayofyear', values = 'mean').reset_index()
var_names = ['Day' +str(i+1) for i in range(366)]
test3.rename(columns = {old:new for old, new in zip(test3.columns[3:], var_names)}, inplace=True)

test4 = Temp_20year[Temp_20year.station.isin(Stations_All_20_years)].query('~TMIN.isnull() & ~TMAX.isnull() & year.between(2017,2021)').assign(diff = lambda x: x.TMAX - x.TMIN)
test4 = test4.assign(TMEAN = lambda x: (x.TMIN + x.TMAX)/2).groupby(['station', 'dayofyear'])['TMEAN'].aggregate(['mean', 'std']).reset_index()
test4 = test4.merge(Chi_locations, how = 'left', left_on = "station", right_on = "id")
test4 = test4.sort_values(['dayofyear', 'name'])
test4 = test4.set_index(['station', 'name', 'geometry']).pivot(columns = 'dayofyear', values = 'mean').reset_index()
var_names = ['Day' +str(i+1) for i in range(366)]
test4.rename(columns = {old:new for old, new in zip(test4.columns[3:], var_names)}, inplace=True)

In [ ]:
a = np.zeros((2500,len(test3.geometry)))
xi = np.linspace(Chi_locations.longitude.min(), -87.52 , 50) #longitude of Illinois-Indiana border
yi = np.linspace(Chi_locations.latitude.min(), Chi_locations.latitude.max(), 50)
xi, yi = np.meshgrid(xi,yi)
xi, yi = xi.flatten(), yi.flatten()
for i in range(len(xi)):
  for j in range(len(test3.geometry)):
    a[i,j] = haversine(test3.geometry[j].coords[0][::-1], (yi[i], xi[i]))
a = np.where(a == 0, 0.0000001, a)
a_weights = 1/(a**2)
b = np.zeros((2500))
for i in range(len(a_weights)):
  b[i] = a_weights[i].sum()
for i in range(len(a_weights)):
  for j in range(len(test3.geometry)):
    a_weights[i,j] /= b[i]
actual_station_distance = np.zeros((8,8))
for i in range(len(test3)):
  for j in range(len(test3)):
    actual_station_distance[i,j] = haversine(test3.geometry[i].coords[0][::-1], test3.geometry[j].coords[0][::-1])
actual_station_distance = np.where(actual_station_distance == 0, 1000000, actual_station_distance)
weights = (1/(actual_station_distance**2))
sum_distance = np.zeros((len(test3)))
for i in range(len(weights)):
  sum_distance[i] = weights[i].sum()
for i in range(len(weights)):
  for j in range(len(weights)):
    weights[i,j] /= sum_distance[i]

In [ ]:
zi = np.zeros((366, 50, 50))
for i in range(366):
  zi[i,:,:] = np.dot(a_weights, test4.iloc[:,i+3] - test3.iloc[:,i+3]).reshape(50,50)

In [ ]:
fig = plt.figure()
ax = plt.axes()
gdf4.plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
Temp_20year_stations.plot(ax = ax, marker = '.', c = 'black')
gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].clip(clip).plot(ax = ax, **{'linewidth':0.06, 'facecolor':'turquoise'})
im = ax.imshow(zi[0], extent = (Chi_locations.longitude.min(), -87.52, Chi_locations.latitude.min(), Chi_locations.latitude.max()), cmap = 'bwr', vmin = -10, vmax = 30)
plt.colorbar(im)

In [ ]:
def init():
  im.set_data(np.zeros((50,50)))
  return
def animate(i):
  ax.clear()
  im.set_array(zi[i])
  gdf4.clip(clip).plot(ax = ax, **{'linewidth':0.05, 'edgecolor':'black', 'facecolor':'none'})
  gdf4[gdf4.NAMELSAD == 'Census Tract 9900'].clip(clip).plot(ax = ax, **{'linewidth':0.06, 'facecolor':'turquoise'})
  ax.imshow(zi[i], cmap = 'bwr', extent = (Chi_locations.longitude.min(), -87.52, Chi_locations.latitude.min(), Chi_locations.latitude.max()), vmin = -10, vmax = 10)
  ax.annotate(f' Day {str(i+1)}' f', Mean Difference In 5 Year Temperature: {(test4.iloc[:,i+3].mean()-test3.iloc[:,i+3].mean()).round(1)}C', xy = (0.5,1.02), xycoords='axes fraction', ha = 'center')
anim2 = FuncAnimation(fig, animate, init_func = init, frames = range(365), interval = 300, repeat = False)

In [ ]:
anim2

##### The difference from 2019-2023 to 2004-2008 is very up and down from day to day since the dataset is small. We can also visualize this trend as a histogram or a line.

In [ ]:
test5 = []
for i in range(len(test3.columns)-4):
    test5.append(test4.iloc[:,i+3].mean()-test3.iloc[:,i+3].mean())

In [ ]:
fig, ax = plt.subplots(**{'figsize':(12,2)})
pd.DataFrame(np.linspace(1,365,365), test5).reset_index().rename({0:'Day', 'index':'5_year_temp_change'}, axis = 1).plot('Day', '5_year_temp_change', ax = ax)
ax.axhline(0, xmin = 0, xmax = 365, c = "black")

In [ ]:
pd.Series(test5).plot.hist(bins = 50)
print(f'Mean Change in 5 year Average Temperature From 2004-2008 to 2019-2023: {pd.Series(test5).mean().round(1)} C')

##### Let's look at all the datatypes and their abbreviations we scraped earlier

In [ ]:
pd.set_option('display.max_rows', None)
results3.groupby('datatype').count()['station'].sort_values(ascending = False).to_frame().reset_index()\
.rename({'station':'count'}, axis = 1).merge(datatypes, how = "left", left_on = 'datatype', right_on = 'id')
#WDF5     22375 #direction of fastest 5-second wind
#WSF5     22385 #Fastest 5-second wind speed
#AWND     22542 #Average wind speed
#WDF2     22561 #Direction of fastest 2-minute wind
#WSF2     22563 #Fastest 2 minute wind speed
#WESD     24929 #Water equivalent of snow on ground
#WESF     35303 #Water equivalent of snowfall

In [ ]:
Average_wind = results3.query('datatype == "AWND"')

In [ ]:
pd.set_option('display.max_rows',10)
Average_wind.station.unique()
#only four stations

In [ ]:
#Let's look at these four stations by name
Chi_locations.query('id.isin(("GHCND:USW00004838", "GHCND:USW00014819", "GHCND:USW00094846", "GHCND:USC00117457"))')

In [ ]:
#just to confirm all times are recorded as 00:00:00
Average_wind.assign(Time = lambda x: x.date.str.extract(r'([0-9]{2}:[0-9]{2}:[0-9]{2})')).groupby('Time').count()

In [ ]:
#Merge the information about the stations from Chi_locations with Average_wind
Average_wind = Chi_locations.query('id.isin(("GHCND:USW00004838", "GHCND:USW00014819", "GHCND:USW00094846", "GHCND:USC00117457"))')[['name', 'id']] \
.merge(Average_wind, left_on = 'id', right_on = 'station').drop('id', axis = 1).assign(name = lambda x: x.name.str.replace(", IL US", "")) \
.drop(['Unnamed: 0', 'datatype', 'attributes'], axis =1).assign(dayofyear = lambda x:pd.to_datetime(x.date).dt.dayofyear, date = lambda x: pd.to_datetime(x.date).dt.date)

In [ ]:
Average_wind_summary = Average_wind.groupby(['name', 'dayofyear'])['value'].aggregate(['mean', 'std'])

##### Let's look at a plot of average wind speed at each station by day of the year

In [ ]:
fig, axs = plt.subplots(2,2, **{'figsize':(12,9)})
for ax, (name, new) in zip(axs.flatten(), Average_wind_summary.reset_index().groupby('name')):
  new.plot('dayofyear', 'mean', ax = ax)
  ax.set_title(str(name), **{'fontsize':9})
  ax.set_xlabel('Day of Year')
  ax.set_ylabel('Wind Speed in m/s')

##### All stations have lower wind speed in the summer. This should exacerbate the Urban Heat Island effect: Per Wikipedia article on Urban Heat Islands "The temperature difference is usually larger at night than during the day, and is most apparent when winds are weak"

##### The daily mean is too granular and results in a very jagged line. Let's smoothe it out a bit by summarizing by week instead of day:

In [ ]:
Weekly_summary = Average_wind.assign(weekofyear = lambda x: x.dayofyear // 7).drop(['date','station', 'dayofyear'], axis=1).groupby(['name', 'weekofyear'])['value'].aggregate(['mean', 'std'])

In [ ]:
fig, axs = plt.subplots(2,2, **{'figsize':(12,9)})
for ax, (name, new) in zip(axs.flatten(), Weekly_summary.reset_index().groupby('name')):
  new.plot('weekofyear', 'mean', ax = ax)
  ax.set_title(str(name), **{'fontsize':9})
  ax.set_ylabel('Wind Speed in m/s')

In [ ]:
Weekly_summary_2 = Weekly_summary.drop('std', axis = 1).reset_index().set_index('weekofyear').pivot(columns = 'name', values = 'mean').reset_index()

In [ ]:
fig, ax = plt.subplots(**{'figsize':(12,9)})
for i in range(len(Weekly_summary_2.columns)-1):
  line, = ax.plot(Weekly_summary_2.weekofyear,Weekly_summary_2.iloc[:,i+1])
  ax.set_ylabel('Wind Speed in m/s')
  line.set_label(Weekly_summary_2.columns[i+1])
  ax.legend()
ax.set_xlabel('Week of the Year')

##### Plotting these lines together makes two trends clear: it is consistently less windy at the Palwaukee airport than the other stations, and the Romeoville station has much higher variance by week of year than the other stations. This may be due to the timing of seasonal changes being more variable from year to year further inland. If we plot each year's data at each station, we would expect a smooth trend.

In [ ]:
#Make a column of average wind measurement each week of each year for each station
Week_yearly_weather = Average_wind.assign(year = lambda x: pd.to_datetime(x.date).dt.year, weekofyear = lambda x: x.dayofyear // 7)\
.drop(['date', 'station', 'dayofyear'], axis =1).groupby(['name', 'weekofyear', 'year']).aggregate("mean").reset_index()\
.set_index('weekofyear').pivot(columns = ['name', 'year'], values = 'value')

In [ ]:
Week_yearly_weather

##### Though upon further inspection the variability in the Romeoville station owes to that station only having 3 years of data from 2004 through 2023 instead of the 20 years of the other stations. Within a given year, the weekly wind averages at other stations are just as volatile from week to week.

In [ ]:
fig, ax = plt.subplots(2,2, **{'figsize': (12,9)})
for i in range(len(Week_yearly_weather.columns.levels[0])-1):
  for j in range(len(Week_yearly_weather.columns.levels[1])):
    ax[i//2, i%2].plot(Week_yearly_weather.reset_index().weekofyear, Week_yearly_weather.loc[:,(Week_yearly_weather.columns.levels[0][i],Week_yearly_weather.columns.levels[1][j])], linewidth = 0.5, c = "blue", **{'alpha':0.5})
    ax[i//2, i%2].set_title(Week_yearly_weather.columns.levels[0][i])
for j in range(17,20):
    ax[1,1].plot(Week_yearly_weather.reset_index().weekofyear, Week_yearly_weather.loc[:,(Week_yearly_weather.columns.levels[0][3],Week_yearly_weather.columns.levels[1][j])], linewidth = 0.5, c = "blue", **{'alpha':0.5})
    ax[1,1].set_title(Week_yearly_weather.columns.levels[0][3])

##### we wouldn't expect a correlation, but we can compare how our approximation errors (too warm/too cold) are related to windspeed with a scatter plot.

In [ ]:
IDW_error_windspeed_comparison = residual_check.set_index('station').T.reset_index().drop('index', axis = 1).merge(asd.columns.to_frame().rename({'date2':'date'}, axis = 1).reset_index()\
.drop('date2', axis = 1), left_index = True, right_index = True).set_index('date').melt(ignore_index = False)\
.query('variable.isin(("GHCND:USW00004838", "GHCND:USW00014819", "GHCND:USW00094846", "GHCND:USC00117457"))').rename({'value':'IDW error'}, axis = 1)\
.reset_index().merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), left_on = ['date', 'variable'], right_on = ['date', 'station'])
IDW_error_windspeed_comparison

##### This plot is consistent with no correlation. The error in our IDW approximation is tied to differences between station like degree of forestation, whereas we are just looking here at windspeeds vs errors at the same station, and even if it may be cooler with high winds, it is likely to be windier and thus cooler at the other neighboring stations we rely on for our IDW approximation.

In [ ]:
plt.scatter(IDW_error_windspeed_comparison['IDW error'], IDW_error_windspeed_comparison.windspeed, s = 0.01)

##### It's a better question to ask if we can find any relation between the temperature drop from TMAX to TMIN on days with higher winds, particularly those in the summer, when the Urban Heat Island effect is at its strongest.

In [ ]:
fig, ax = plt.subplots(1,2)
Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()]\
.assign(TDrop = lambda x: x.TMAX - x.TMIN).query('month.isin((6,7,8))').query('TMAX > 25')\
.merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), \
       left_on = ['date2', 'station'], right_on = ['date', 'station'])\
.plot.scatter('windspeed', 'TDrop', ax = ax[0], s = 0.1, **{'alpha':0.1})
Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()]\
.assign(TDrop = lambda x: x.TMAX - x.TMIN).query('month.isin((6,7,8))').query('TMAX > 25')\
.merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), \
       left_on = ['date2', 'station'], right_on = ['date', 'station']).groupby('windspeed')[['windspeed', 'TDrop']].\
aggregate('mean').plot.scatter('windspeed', 'TDrop', ax = ax[1])
ax[0].set_title('All Points')
ax[1].set_title('Average Temperature Drop by Wind Speed')

##### a slight relationship where higher windspeed means a small temperature drop. The higher spread of average temperature drop at higher wind speeds is due to far fewer data points allowing for more variance.

##### one shortcoming of the previous plot is that by aggregating across June, July, and August we treat these months as identical, when there's a clear trend toward larger temperature drops in June than in July and August.

In [ ]:
fig, ax = plt.subplots()
Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()]\
.assign(TDrop = lambda x: x.TMAX - x.TMIN).query('month.isin((6,7,8))').query('TMAX > 25')\
.merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), \
       left_on = ['date2', 'station'], right_on = ['date', 'station'])\
.plot.scatter('dayofyear', 'TDrop', s = 0.1, ax = ax)
plt.vlines(182, 0, 25, color = 'black', **{'linewidth':0.2})
plt.text(160, 23, 'June')
plt.vlines(213, 0, 25, color = 'black', **{'linewidth':0.2})
plt.text(191, 23, 'July')
plt.text(222, 23, 'August')
#add a rolling 3 day mean Temperature Drop to the plot:
Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()]\
.assign(TDrop = lambda x: x.TMAX - x.TMIN).query('month.isin((5,6,7,8))').query('TMAX > 25')\
.merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), \
       left_on = ['date2', 'station'], right_on = ['date', 'station']).groupby('dayofyear')['TDrop']\
.aggregate('mean').rolling(window = 3, min_periods = 3).mean().reset_index().query('dayofyear > 151')\
.plot('dayofyear', 'TDrop', ax = ax, c = 'black')

##### Indeed we can see that the rolling temperature drop from the maximum temperature to the minimum temperature is highest in the spring and fall, slightly lower in the summer, and much lower in the winter. This makes sense given the length of day: we'd expect the drop from max to min temperature to be greatest when days and nights are roughly equally long, and there is time for the sun to heat up the air and buildings during the day and to cool off at night. The more humid hot air in summer takes longer to cool at night than dry air, and the longer days expose large buildings to more sunlight that would take more time to be released. In winter the days are too short and the sun to low in the sky for the days to get much warmer than the nights.

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim(0,12)

Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()]\
.assign(TDrop = lambda x: x.TMAX - x.TMIN).merge(Average_wind.drop('dayofyear', axis = 1).rename({'value':'windspeed'}, axis =1), \
       left_on = ['date2', 'station'], right_on = ['date', 'station']).groupby('dayofyear')['TDrop']\
.aggregate('mean').rolling(window = 7, min_periods = 3).mean().reset_index().plot('dayofyear', 'TDrop', ax = ax, c = 'black', title = "Average Drop from Max to Min Temperature", legend = False)


##### Instead we compare within the day of the year: if it's day 150 of the year is the temperature drop greater than the average temperature drop on day 150 if it's windier than average?

In [ ]:
Temp_Wind_Drop = Temp_20year[~Temp_20year.TMIN.isnull() & ~ Temp_20year.TMAX.isnull()].assign(TDrop = lambda x: x.TMAX - x.TMIN).groupby('dayofyear')['TDrop'].aggregate(['mean'])\
.merge(Average_wind.groupby('dayofyear')['value'].mean(), left_index = True, right_index = True).rename({'value':'Average_Wind_Speed', 'mean':'Average_Temperature_Drop'}, axis  = 1)\
.reset_index().merge(Average_wind.rename({'value':'windspeed'}, axis=1), how = 'right', left_on = 'dayofyear', right_on = 'dayofyear')\
.merge(Temp_20year[~Temp_20year.TMAX.isnull() & ~Temp_20year.TMIN.isnull()], left_on = ['date', 'station', 'dayofyear'], right_on = ['date2', 'station', 'dayofyear']).assign(TDrop = lambda x: x.TMAX - x.TMIN)\
.sort_values('dayofyear').assign(TDrop_Difference = lambda x: x.TDrop - x.Average_Temperature_Drop, Wind_Difference = lambda x: x.windspeed - x.Average_Wind_Speed)

##### In the summer there is a negative correlation between higher wind speeds than usual on that day of the year and temperature drop, fitting the trend we saw above with higher winds being correlated with smaller temperature drops.

In [ ]:
Temp_Wind_Drop.query('month.isin((6,7,8))').plot.scatter('Wind_Difference', 'TDrop_Difference', s = 0.1)

##### Plotting the delta in TMIN compared to the average TMIN on a given day of the summer suggests that this mostly comes from a slight relationship between higher winds and a higher minimum temperature in the summer, whereas the TMAX seems uncorrelated with change in wind speed

In [ ]:
fig, axs = plt.subplots(1,2, **{'figsize': (12, 6)})
Temp_Wind_Drop.query('month.isin((6,7,8))').groupby('dayofyear')['TMIN'].aggregate(['mean']).rename({'mean': 'Average_TMIN'}, axis=1)\
.merge(Temp_Wind_Drop.query('month.isin((6,7,8))'), left_index = True, right_on = 'dayofyear').assign(TMIN_Difference = lambda x: x.TMIN - x.Average_TMIN)\
.plot.scatter('Wind_Difference', 'TMIN_Difference', s = 0.1, ax = axs[0])
Temp_Wind_Drop.query('month.isin((6,7,8))').groupby('dayofyear')['TMAX'].aggregate(['mean']).rename({'mean': 'Average_TMAX'}, axis=1)\
.merge(Temp_Wind_Drop.query('month.isin((6,7,8))'), left_index = True, right_on = 'dayofyear').assign(TMAX_Difference = lambda x: x.TMAX - x.Average_TMAX)\
.plot.scatter('Wind_Difference', 'TMAX_Difference', s = 0.1, ax = axs[1])

##### Verify the fastest wind speed is correlated with the average wind speed

In [ ]:
results3.query('datatype == "WSF5"').assign(date = lambda x: pd.to_datetime(x.date).dt.date).drop(['Unnamed: 0', 'attributes'], axis = 1).rename({'value': 'fastest_5_min_wind'}, axis = 1)\
.merge(Average_wind, left_on = ['date', 'station'], right_on = ['date', 'station']).rename({'value':'Average_Wind_speed'}, axis = 1).plot.scatter('Average_Wind_speed', 'fastest_5_min_wind', s = 0.1)

##### The relationship between wind and temperature is almost certainly a much more complicated one than higher winds cooling a large city resulting in lower temperatures or vice versa. Winds blow from areas of high pressure to low pressure, and pressure is lower in the hotter months as the warm, less dense air rises.

In [ ]:
results3.query('datatype == "WDF2"')

##### let's look at average station pressure to compare to temperature

In [ ]:
pd.set_option('display.max_rows', 100)
Average_station_pressure = results3.query('datatype == "ASTP"').assign(month = lambda x: pd.to_datetime(x.date).dt.month, \
dayofyear = lambda x: pd.to_datetime(x.date).dt.dayofyear,date = lambda x: pd.to_datetime(x.date).dt.date, \
).drop(['Unnamed: 0', 'attributes'], axis = 1)


##### Pressure is slightly higher on average in winter than summer, so we will continue to focus on summer temperatures:

In [ ]:
fig, axs = plt.subplots(1,2, **{'figsize':(12,6)})
Average_station_pressure.query('month.isin((6,7,8))').hist('value', ax = axs[0], bins = 20)
axs[0].set_title("Summer Pressure")
Average_station_pressure.query('month.isin((12,1,2))').hist('value', ax = axs[1], bins = 20)
axs[1].set_title("Winter Pressure")

##### The only location here where the pressure is measured is O'Hare International

In [ ]:
Chi_locations.query('id == "GHCND:USW00094846"')

##### Higher pressure is correlated with lower TMIN. Cold air is denser than warmer air, so pressure is higher when temperatures are cool.

In [ ]:
Average_station_pressure.merge(Temp_20year, left_on = ['station', 'date', 'month', 'dayofyear'], right_on = ['station', 'date2', 'month', 'dayofyear'])\
.query('month.isin((6,7,8))').assign(standardized_pressure = lambda x: (x.value - x.value.mean())/x.value.std(), TMIN_jit = lambda x: x.TMIN + random.uniform(0,.1))\
.plot.scatter('standardized_pressure', 'TMIN_jit', s = 0.1)

##### Slight trend where higher pressure is correlated with lower wind speed, as expected.

In [ ]:
Average_station_pressure.merge(Average_wind, left_on = ['station', 'date', 'dayofyear'], right_on = ['station', 'date',  'dayofyear'])\
.rename({'value_x':'Pressure', 'value_y':'Wind_speed'}, axis =1).query('month.isin((6,7,8))').plot.scatter('Pressure', 'Wind_speed', s = 0.1)

##### So while we started out looking to see how winds are correlated with the Urban Heat Island effect, it is not straightforward to isolate this from the greater correlation between higher winds and higher temperatures. We can try again

In [ ]:
Wind_direction = results3.query('datatype.isin(("WDF5", "WSF5", "WDF2", "WSF2"))').drop(['attributes', 'Unnamed: 0'], axis = 1)\
.assign(month = lambda x: pd.to_datetime(x.date).dt.month, dayofyear = lambda x: pd.to_datetime(x.date).dt.dayofyear, \
date = lambda x: pd.to_datetime(x.date).dt.date).set_index(['date', 'month', 'dayofyear', 'station'])\
.pivot(columns = 'datatype', values = 'value').reset_index()
fig, axs = plt.subplots(1,2)
Wind_direction.query('month.isin((12,1,2))').hist('WDF5', ax = axs[0], bins = 30)
axs[0].set_title("Winter Wind Direction")
Wind_direction.query('month.isin((6,7,8))').hist('WDF5', ax = axs[1], bins = 30)
axs[1].set_title("Summer Wind Direction")

##### Wind direction with north given by 0/360, East given by 90, South 180, West 270. Winter winds tend very strongly to blow in a southwest direction, while summer winds tend to blow in a range between North and East, or a wide range of west directions

In [ ]:
pd.set_option('display.max_columns', None)
Summer_Wind_Temp = Wind_direction.merge(Temp_Wind_Drop, left_on = ['date', 'dayofyear', 'month', 'station'], right_on = ['date', 'dayofyear', 'month', 'station']).drop(['TOBS', 'dayofmonth'], axis = 1)\
.query('month.isin((6,7,8))')
fig, axs = plt.subplots(1,2, sharex = True, sharey = True)
Summer_Wind_Temp.query('WDF5 <= 180').plot.scatter('WSF5', 'TDrop', ax = axs[0])
Summer_Wind_Temp.query('WDF5 > 180').plot.scatter('WSF5', 'TDrop', ax = axs[1])
axs[0].set_title('East Wind Summer Temperature Drop', **{'fontsize': 9})
axs[1].set_title('West Wind Summer Temperature Drop', **{'fontsize': 9})
axs[0].text(20, 23, "Average Drop:" + str(Summer_Wind_Temp.query('WDF5 <= 180')['TDrop'].mean().round(1)))
axs[1].text(20, 23, "Average Drop:" + str(Summer_Wind_Temp.query('WDF5 > 180')['TDrop'].mean().round(1)))

##### This is a crude approximation of the average direction of the wind - categorizing all degrees under 180 as a wind blowing east and all over 180 as blowing west - and the direction of the wind is likely to change within the same day, and the most common or average direction of the wind may be quite different from the direction of the direction of the fastest 5 minute burst, but it follows a mechanism that we would expect: the west wind blowing from the cooler lake over the city cools the city more than the east wind from the hotter land.

In [ ]:
Wind_direction.groupby('date').mean([['WDF2', 'WDF5', 'WSF2', 'WSF5']])

In [ ]:
Degree_diff = Wind_direction[['date', 'WDF5']].groupby('date').max(['WDF5']).merge(Wind_direction[['date', 'WDF5']].groupby('date').min(['WDF5']), left_index = True, right_index = True).assign(diff = lambda x: x.WDF5_x - x.WDF5_y)

In [ ]:
Degree_diff['diff'] = np.where(Degree_diff['diff'] > 180, 360 - Degree_diff['diff'], Degree_diff['diff'])

##### In 88% of cases, the difference between the three stations wind direction readings is under 90 degrees, so generally the same direction:

In [ ]:
Degree_diff.query('diff < 90')['diff'].count()/Degree_diff.count()

In [ ]:
#Histogram of differences between wind direction readings
Degree_diff.hist('diff', bins = 30)

##### Compare the Northerly Island TDrop with the wind direction at the nearest station with wind measurements, Midway.

In [ ]:
NI_MW = Wind_direction.query('station == "GHCND:USW00014819"').merge(Temp_20year.query('station == "GHCND:USC00111550"'), left_on = ['date', 'month', 'dayofyear'], right_on = ['date2', 'month', 'dayofyear'])\
.query('~TMAX.isnull() & ~TMIN.isnull() & ~WSF5.isnull()').assign(TDrop = lambda x: x.TMAX - x.TMIN).query('month.isin((6,7,8))')

##### For a station right on the lake, the trend is predictably even more pronounced. The winds coming off the lake don't warm up over the city at all, while the winds blowing from the east come from the part of the city with the largest skyscrapers, most prone to urban heating.

In [ ]:
fig, axs = plt.subplots(1,2, sharey = True, **{'figsize': (12,6)})
NI_MW.query('WDF5 <= 180').plot.scatter('WSF5', 'TDrop', s = 0.2, ax = axs[0])
NI_MW.query('WDF5 > 180').plot.scatter('WSF5', 'TDrop', s = 0.2, ax = axs[1])
axs[0].text(18, 16, 'Average Temp Drop:' + str(NI_MW.query('WDF5 <= 180')['TDrop'].mean().round(1)))
#axs[0].text(18, 15, 'Correlation: ' + str(NI_MW.query('WDF5 <= 180')[['TDrop', 'WSF5']].corr().iloc[0,1].round(2)))
axs[1].text(18, 16, 'Average Temp Drop:' + str(NI_MW.query('WDF5 > 180')['TDrop'].mean().round(1)))
#axs[1].text(18, 15, 'Correlation: ' + str(NI_MW.query('WDF5 > 180')[['TDrop', 'WSF5']].corr().iloc[0,1].round(2)))
axs[0].set_title('East Wind Temperature Drop')
axs[1].set_title('West Wind Temperature Drop')

##### We look for a station with temperature values at roughly the same latitude as O'Hare but far enough outside the city to be free of the Urban Heat Island effect.

In [ ]:
fig, ax = plt.subplots(figsize = (9,9))
gpd.GeoDataFrame(All_Illinois_stations, geometry = gpd.points_from_xy(All_Illinois_stations.longitude, All_Illinois_stations.latitude))\
.plot(**{'markersize': 0.5}, ax = ax)
gdf3.plot(**{'facecolor':'None', 'linewidth':0.1, 'alpha':0.5}, ax = ax)

In [ ]:
DeKalb = DeKalb.set_index('date').pivot(columns = 'datatype', values = 'value')[['PRCP', 'TMAX', 'TMIN', 'TOBS', 'WDMV']].reset_index()

In [ ]:
DeKalb = DeKalb.assign(TDrop = lambda x: x.TMAX - x.TMIN, date = lambda x: pd.to_datetime(x.date).dt.date)\
.rename({'TMAX': 'TMAX_DK', 'TMIN': 'TMIN_DK', 'TDrop': 'TDrop_DK'}, axis =1)

In [ ]:
DK_OH_comparison = Average_station_pressure.merge(Temp_20year.query('station == "GHCND:USW00094846"')\
,left_on = ['date'], right_on = ['date2']).merge(DeKalb, left_on = 'date', right_on = 'date')\
.assign(TDrop = lambda x: x.TMAX - x.TMIN, TMAX_diff = lambda x: x.TMAX - x.TMAX_DK, \
        TMIN_diff = lambda x: x.TMIN - x.TMIN_DK, TDrop_DK = lambda x: x.TMAX_DK - x.TMIN_DK)

In [ ]:
DK_OH_comparison = DK_OH_comparison.drop(['TOBS_x', 'TOBS_y','PRCP'], axis =1)\
.rename({'value': 'Average_Pressure', 'TMAX': 'TMAX_OH', 'TMIN': 'TMIN_OH'}, axis = 1)

In [ ]:
fig, axs = plt.subplots(1,2, **{'figsize':(12,6)}, sharey = True, sharex = True)
DK_OH_comparison.query('month_x.isin((6,7,8))').merge(Wind_direction.query('station == "GHCND:USW00004838"'), left_on = ['date'], right_on = ['date']).query('WDF5 <= 180')\
.plot.scatter('TMAX_diff', 'TMIN_diff', ax = axs[0])
DK_OH_comparison.query('month_x.isin((6,7,8))').merge(Wind_direction.query('station == "GHCND:USW00004838"'), left_on = ['date'], right_on = ['date']).query('WDF5 > 180')\
.plot.scatter('TMAX_diff', 'TMIN_diff', ax = axs[1])

In [ ]:
print("Difference between OHare and DeKalb Temperature Drop with East Wind:" + str(DK_OH_comparison.query('month_x.isin((6,7,8))').merge(Wind_direction, left_on = ['station_x', 'date', 'month_x', 'dayofyear_x'], right_on = ['station', 'date', 'month', 'dayofyear']).assign(TDrop_diff = lambda x: x.TDrop - x.TDrop_DK)\
.query('WDF5 <= 180')['TDrop_diff'].mean().round(2)))
print("Difference between OHare and DeKalb Temperature Drop with West Wind:" + str(DK_OH_comparison.query('month_x.isin((6,7,8))').merge(Wind_direction, left_on = ['station_x', 'date', 'month_x', 'dayofyear_x'], right_on = ['station', 'date', 'month', 'dayofyear']).assign(TDrop_diff = lambda x: x.TDrop - x.TDrop_DK)\
.query('WDF5 > 180')['TDrop_diff'].mean().round(2)))

##### So the temperature drop at the inland DeKalb station tends to be lower relative to the temperature drop at O'Hare when there is a West Wind than when there is an East Wind, as we'd expect: the effect of a cool wind off the lake will be much less inland after it has warmed up over land and the city.

In [ ]:
pd.set_option('display.max_rows', None)
All_Illinois_stations.groupby('id').first()

##### One potential shortcoming with this approach of comparing wind direction and temperature drop is that the lowest temperature in Chicago almost always comes after midnight in the early hours of the morning while the maximum temperature comes in the afternoon. So if the measurements are organized from midnight to midnight on one calendar day as they apparently are, we can compare the rise in temperature in one row of data, but if we want to look at a drop in temperature due to wind, we should consider comparing the maximum temperature to the next day's minimum temperature. Of course, we would also be interested in the direction of the wind from the afternoon to the early morning, and this may be different from the direction of the strongest 5 minute burst in the data.

##### We've reached the point where we need more granular data to answer our questions. 

In [ ]:
pd.set_option('display.max_rows', 10)
OHare_hourly_summer = OHare_hourly[['Station_name', 'Year', 'Month', 'Day', 'Hour', 'temperature', 'station_level_pressure', 'wind_direction', 'wind_speed']]\
.assign(temp_change = lambda x: x.temperature - x.temperature.shift(1)).assign(TMax = OHare_hourly.groupby(['Day', 'Month', 'Year'])\
 ['temperature'].transform('max'), Index_of_max = (OHare_hourly.groupby(['Day', 'Month', 'Year'])['temperature'].transform('idxmax')), \
Hour_of_max = lambda x: OHare_hourly.Hour[OHare_hourly.index == x.Index_of_max]).query('Month.isin((6,7,8))')

In [ ]:
OHare_hourly_summer['Hour_of_max'] = OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])['Hour_of_max'].bfill().ffill()

##### Look at hour of maximum temperature. According to documentation, "Each observation is also indexed to a single Year, Month, Day, Hour and Minute in Coordinated Universal Time (UTC)."

In [ ]:
OHare_hourly_summer.hist('Hour_of_max', bins = 24)

##### The distribution of hours from 15 to 22 looks reasonable, but the peak at 0 hours (around 7 PM local) is unusual and suggests a daily summary

##### There is an increasing number of observations per year over time with peak temperature at 00:00: 


In [ ]:
OHare_hourly_summer.query('Hour_of_max == 0').hist('Year')

In [ ]:
OHare_hourly_summer.query('Hour_of_max == 0 & Hour == 0').groupby(['Year', 'Month', 'Day'])['Hour'].count().reset_index().rename({'Hour':'Count_by_hour'}, axis = 1).groupby('Count_by_hour').count()['Day']

##### Many days with more than 1 measurement per day

In [ ]:
pd.set_option('display.max_rows', None)
OHare_hourly_summer.groupby(['Year', 'Month', 'Day']).count().reset_index().groupby('Hour_of_max').count()['Year'].sort_values(ascending = False)

##### So a good number of summer observations with more or fewer than 24 measurements per day

In [ ]:
pd.set_option('display.max_rows', 100)
OHare_hourly.groupby(['temperature_Source_Station_ID', 'Hour']).count()

##### We see that there are three different temperature source stations. "999999-94846" has few measurements in 3 hour increments. "ICAO-KORD" has some measurements concentrated in 6 hour increments. "725300-94846" has three times as many measurements and roughly equally distributed by hour; we will focus on this station.

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (12,6))
OHare_hourly.query('temperature_Source_Station_ID == "725300-94846"').hist('Year', ax = axs[0])
OHare_hourly.query('temperature_Source_Station_ID == "ICAO-KORD"').hist('Year', ax = axs[1])
axs[0].set_title("Station 725300-94846")
axs[1].set_title("Station ICAO-KORD")

##### It looks like the station ICAO-KORD was used in the 70s and less before or since. Let's try again to see if the distribution of the hour of the maximum temperature looks better with just this one station.

In [ ]:
OHare_hourly_summer = OHare_hourly[['Station_name', 'Year', 'Month', 'Day', 'Hour', 'temperature', 'station_level_pressure', 'wind_direction', 'wind_speed', 'temperature_Source_Station_ID']]\
.assign(temp_change = lambda x: x.temperature - x.temperature.shift(1)).query('temperature_Source_Station_ID == "725300-94846" & Month.isin((6,7,8))')

In [ ]:
OHare_hourly_summer = OHare_hourly_summer.assign(TMax = OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])\
 ['temperature'].transform('max'), Index_of_max = (OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])['temperature'].transform('idxmax')), \
Hour_of_max = lambda x: OHare_hourly_summer.Hour[OHare_hourly_summer.index == x.Index_of_max])

In [ ]:
OHare_hourly_summer['Hour_of_max'] = OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])['Hour_of_max'].bfill().ffill()

In [ ]:
OHare_hourly_summer.hist('Hour_of_max', bins = 24)

In [ ]:
OHare_hourly_summer.query('Hour_of_max == 0 & Hour.isin((0,1,2))').head(48)

In [ ]:
OHare_hourly[[col for col in OHare_hourly.columns if 'Quality_Code'  not in col]].columns

In [ ]:
OHare_hourly_summer.columns

In [ ]:
OHare_hourly.iloc[:,[i for i,x in enumerate(OHare_hourly.columns.str.contains('sky')) if x]]

In [ ]:
OHare_hourly_summer.query('Hour_of_max == 0 & Hour == 0').groupby(['Day', 'Month', 'Year']).count()['Station_name'].reset_index().groupby('Station_name').count()

##### not many observations have multiple values of hour 0 though

In [ ]:
OHare_hourly_summer.assign(hour_indicator = np.where(OHare_hourly_summer.Hour_of_max == 0.0, 1, 0)).query('Hour_of_max == Hour').merge(OHare_hourly[[col for col in OHare_hourly.columns if col not in OHare_hourly_summer.columns]], left_index = True, right_index = True)\
.groupby(['hour_indicator','temperature_Report_Type']).count()['Station_name']

In [ ]:
OHare_hourly_summer.assign(hour_indicator = np.where(OHare_hourly_summer.Hour_of_max == 0.0, 1, 0)).query('Hour_of_max != Hour').merge(OHare_hourly[[col for col in OHare_hourly.columns if col not in OHare_hourly_summer.columns]], left_index = True, right_index = True)\
.groupby(['hour_indicator','temperature_Report_Type']).count()['Station_name']

##### If we compare this query with the last we look to see if there is a difference in the report type at hour 0 vs the other 23 hours of the day, especially in hour_indicator category 1 where the max temperature is recorded at hour 0. We might naively expect 23 times more rows for each report type with this query than with the previous query if the report types were distributed randomly and weren't connected to the spike in max temperature recordings happening at hour 0. A ratio far less than 23 more results in this query than in the last for the same category of report type is suspicious with this assumption.

##### Indeed, many categories show this ratio (or greater), but the categories FM12-Synop, SYMT-Synop, and SYSA-Synop are all well under this ratio at about 3 to 1. 

##### We do a web search for these codes and find that synoptic observations are those recorded every 6 hours by the NOAA, at 00:00, 06:00, 12:00, and 18:00 UTC. This fits with the spike at hour 0: observations at 7 PM local only need to be hotter than at 1 PM or 1 AM to be the daily max temperature.

In [ ]:
OHare_hourly_summer.merge(OHare_hourly[[col for col in OHare_hourly.columns if col not in OHare_hourly_summer.columns]], left_index = True, right_index = True)\
.query('temperature_Report_Type.str.contains("Synop")').sort_values(['Year','Month', 'Day'])

##### The synoptic hour reports seem to extend only from 1980 to 2005. As expected, a report is made every 6 hours.

##### Let's try to create our DataFrame again but investigate which report types have 24 reports in a day


In [ ]:
OHare_hourly_summer = OHare_hourly[['Station_name', 'Year', 'Month', 'Day', 'Hour', 'temperature', 'station_level_pressure', 'wind_direction', 'wind_speed']]\
.assign(temp_change = lambda x: x.temperature - x.temperature.shift(1)).assign(TMax = OHare_hourly.groupby(['Day', 'Month', 'Year'])\
 ['temperature'].transform('max'), Index_of_max = (OHare_hourly.groupby(['Day', 'Month', 'Year'])['temperature'].transform('idxmax')), \
Hour_of_max = lambda x: OHare_hourly.Hour[OHare_hourly.index == x.Index_of_max]).query('Month.isin((6,7,8))')
OHare_hourly_summer['Hour_of_max'] = OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])['Hour_of_max'].bfill().ffill()

In [ ]:
OHare_hourly[['Station_name', 'Year', 'Month', 'Day', 'Hour', 'temperature', 'station_level_pressure', 'wind_direction', 'wind_speed', 'temperature_Source_Station_ID', 'temperature_Report_Type']]\
.assign(temp_change = lambda x: x.temperature - x.temperature.shift(1), Daily_Observations = OHare_hourly.groupby(['Year', 'Month', 'Day', 'temperature_Source_Station_ID', 'temperature_Report_Type'])\
    ['temperature'].transform('count')).assign(TMax = OHare_hourly.groupby(['Day', 'Month', 'Year'])\
 ['temperature'].transform('max'), Index_of_max = (OHare_hourly.groupby(['Day', 'Month', 'Year'])['temperature'].transform('idxmax')), \
Hour_of_max = lambda x: OHare_hourly.Hour[OHare_hourly.index == x.Index_of_max]).query('Month.isin((6,7,8))').query('Daily_Observations >= 20').query('Hour_of_max == 0.0').groupby('Year').count()

##### Finally, we have a way to limit the rows with a max temperature at hour 0: separate results into groups by combination of the source station ID and report type, and years after 1980 have very few rows with max temperature at hour 0 within these groups.

##### Let's restrict ourselves to summer observations

In [ ]:
OHare_hourly_summer = OHare_hourly[['Station_name', 'Year', 'Month', 'Day', 'Hour', 'temperature', 'station_level_pressure', 'wind_direction', 'wind_speed', \
                                    'temperature_Source_Station_ID', 'temperature_Report_Type', 'sky_cover_1', 'sky_cover_2', 'sky_cover_3','precipitation', 'relative_humidity']]\
.assign(temp_change = lambda x: x.temperature - x.temperature.shift(1), Daily_Observations = OHare_hourly.groupby(['Year', 'Month', 'Day', 'temperature_Source_Station_ID', 'temperature_Report_Type'])\
    ['temperature'].transform('count')).assign(TMax = OHare_hourly.groupby(['Day', 'Month', 'Year'])\
 ['temperature'].transform('max'), Index_of_max = (OHare_hourly.groupby(['Day', 'Month', 'Year'])['temperature'].transform('idxmax')), \
Hour_of_max = lambda x: OHare_hourly.Hour[OHare_hourly.index == x.Index_of_max]).query('Month.isin((6,7,8))').query('Daily_Observations >= 20 & Year > 1980')

In [ ]:
OHare_hourly_summer['Hour_of_max'] = OHare_hourly_summer.groupby(['Day', 'Month', 'Year'])['Hour_of_max'].bfill().ffill()

##### We can be more confident that the data reflect genuine hourly observations. We change from UTC to local time to make the cutoff of the peak temperature look more natural. Since we are only looking at summer results, we don't need to account for Daylight Savings Time.

In [ ]:
fig, ax = plt.subplots()
OHare_hourly_summer.assign(Hour_of_max = lambda x: np.where(x.Hour_of_max > 5, x.Hour_of_max - 5.0, x.Hour_of_max + 19.0)).hist('Hour_of_max', bins = 24, range = (0,24), ax = ax)
plt.vlines(12, 0, 25000, color = 'black')
ax.text(11, 25000, 'Noon')
ax.set_title('Number of Times Daily Max Temperature Occurred at Hour')

In [ ]:
#create a timedelta column to control for gaps between measurements
OHare_hourly_summer = OHare_hourly_summer.assign(datetime = pd.to_datetime(OHare_hourly_summer[['Year', 'Month', 'Day', 'Hour']]), timedelta = lambda x: (x.datetime - x.datetime.shift(1))/np.timedelta64(1, 'h'))

In [ ]:
#get a sense of range of pressure values
OHare_hourly_summer.hist('station_level_pressure', bins = 30)

In [ ]:
#Explore distribution of 24 hour changes in pressure
OHare_hourly_summer = OHare_hourly_summer.assign(one_day_difference = lambda x: x.datetime - np.timedelta64(1, 'D')).reset_index()

In [ ]:
OHare_hourly_summer['tf_hour_pressure_difference'] = [(OHare_hourly_summer.station_level_pressure[OHare_hourly_summer.one_day_difference[x] == OHare_hourly_summer.datetime]).min() for x in range(len(OHare_hourly_summer))]

In [ ]:
OHare_hourly_summer['tf_hour_pressure_difference'] = OHare_hourly_summer.station_level_pressure - OHare_hourly_summer['tf_hour_pressure_difference'] 

In [ ]:
fig, ax = plt.subplots()
OHare_hourly_summer.hist('tf_hour_pressure_difference', bins = 30, ax = ax)
ax.set_title('Counts of 24 Hour Change in Pressure (in Millibars)')

In [ ]:
#also make a column for the pressure drop from previous observation
OHare_hourly_summer['pressure_change'] = OHare_hourly_summer.station_level_pressure - OHare_hourly_summer.station_level_pressure.shift(1)
OHare_hourly_summer = OHare_hourly_summer.assign(pressure_change_delta = lambda x: x.pressure_change / x.timedelta)

In [ ]:
OHare_hourly_summer.groupby('sky_cover_1').count()

##### Values are measured in oktas with accompanying acronyms. Higher numbers mean higher percentage of clouds. 'BKN' = "Broken", 'CLR' = "Clear", 'FEW' = "Few", "OVC" = Overcast", 'SCT' = "Scattered", 'X' is 10, 

In [ ]:
OHare_hourly_summer['sky_cover_1'] = np.where(OHare_hourly_summer['sky_cover_1'] == "X", 10, OHare_hourly_summer['sky_cover_1'])

In [ ]:
OHare_hourly_summer[['sky_cover_1', 'sky_cover_2', 'sky_cover_3']] = OHare_hourly_summer[['sky_cover_1', 'sky_cover_2', 'sky_cover_3']].replace(regex=True,to_replace=r'\D',value=r'')

In [ ]:
OHare_hourly_summer[['sky_cover_1', 'sky_cover_2', 'sky_cover_3']] = OHare_hourly_summer[['sky_cover_1', 'sky_cover_2', 'sky_cover_3']].apply(pd.to_numeric)

##### Although the relationship between pressure and temperature is relative to neighboring pressure values and can't be reduced so simply to a two dimensional graph,  we can observe a general trend where higher pressure equals few clouds. We jitter the cloud cover measurements to make this more apparent.

##### Although measurements of 2 occur at a very wide range of pressures, we can see that measurements of 0 - completely clear skies - are centered at higher pressure readings than the measurements of cloud cover 4 or cloud cover 8.


In [ ]:
OHare_hourly_summer.assign(Cloud_cover = OHare_hourly_summer['sky_cover_1'] + np.random.normal(0,0.5,len(OHare_hourly_summer))).plot.scatter('station_level_pressure', 'Cloud_cover', s = 0.1)

##### Except for the very small sample of readings with cloud cover 10, the mean station level pressure with each sky cover reading makes the same relationship clear:

In [ ]:
OHare_hourly_summer.groupby('sky_cover_1')['station_level_pressure'].mean()

##### Similarly, we can see that lower wind speeds are correlated with higher pressure

In [ ]:
OHare_hourly_summer.assign(pressure_change = OHare_hourly_summer.pressure_change + np.random.normal(0,0.1, len(OHare_hourly_summer)), pressure_drop_delta = lambda x: x.pressure_change / x.timedelta, 
                                                           wind_speed_jitter = OHare_hourly_summer.wind_speed + np.random.normal(0,0.2, len(OHare_hourly_summer))
                                                           ).plot.scatter('station_level_pressure', 'wind_speed_jitter', s = 0.1, alpha = 0.1)

##### Rounding wind speed to the nearest integer and finding the average pressure at a given wind speed also shows higher winds at lower pressure. Again the highest wind speeds have very few observations and aren't reliable.

In [ ]:
OHare_hourly_summer.assign(wind_speed = lambda x: x.wind_speed.round(0)).groupby('wind_speed')[['station_level_pressure']].mean()\
.rename({'station_level_pressure': 'Mean_Pressure'}, axis = 1).reset_index().plot.scatter('wind_speed', 'Mean_Pressure')

##### Although a rise in pressure is generally associated with a more negative change in temperature, it looks like a rise in temperature on average is also accompanied by a slight rise in pressure. While pressure usually drops with higher temperature because the warmer air molecules expand in volume, the pressure may rise due to greater mass. One possibility is that the humidity rises more than the volume of the air - increasing the mass and the pressure despite the rise in temperature.

In [ ]:
OHare_hourly_summer.assign(temp_change_custom = lambda x: (x.temp_change * 2).round(0)/2).groupby('temp_change_custom')['pressure_change'].mean().reset_index().plot.scatter('temp_change_custom', 'pressure_change')

##### We make a quantile plot of these values to see if the mean scatterplot is representative and how dispersed values are.

In [ ]:
pressure_temp_quantiles = pd.DataFrame(np.nanquantile(OHare_hourly_summer.assign(temp_change_custom = lambda x: (x.temp_change * 2).round(0)/2)[['temp_change_custom', 'pressure_change']]\
.pivot(columns = 'temp_change_custom', values = 'pressure_change'), np.arange(0.1,1,0.1), axis = 0))

In [ ]:
pressure_temp_quantiles.columns = OHare_hourly_summer.assign(temp_change_custom = lambda x: (x.temp_change * 2).round(0)/2)[['temp_change_custom', 'pressure_change']]\
.pivot(columns = 'temp_change_custom', values = 'pressure_change').columns

In [ ]:
pressure_temp_quantiles = pressure_temp_quantiles.T.reset_index()

In [ ]:
a = mplPolygon(list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,9])) + list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,1]))[::-1], **{'fc':'red', 'alpha':0.1})
b = mplPolygon(list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,8])) + list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,2]))[::-1], **{'fc':'red', 'alpha':0.3})
c = mplPolygon(list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,7])) + list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,3]))[::-1], **{'fc':'red', 'alpha':0.5})
d = mplPolygon(list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,6])) + list(zip(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,4]))[::-1], **{'fc':'red', 'alpha':0.7})

fig,ax = plt.subplots()
ax.set_xlim(-12,9)
ax.set_ylim(-5,5)
ax.add_patch(a)
ax.add_patch(b)
ax.add_patch(c)
ax.add_patch(d)
ax.plot(pressure_temp_quantiles['temp_change_custom'], pressure_temp_quantiles.iloc[:,5], c = 'darkred')

In [ ]:
OHare_hourly_summer.assign(temp_change_custom = lambda x: (x.temp_change * 2).round(0)/2)[['temp_change_custom', 'pressure_change']]\
.pivot(columns = 'temp_change_custom', values = 'pressure_change').columns[1:36]

##### Use the Tetens equation to approximate the saturation vapor pressure of water at a given temperature, the pressure at which water is in thermodynamic equilibrium with its condensed state, and then the actual vapor pressure using relative humidity, as well as the absolute humidity: https://www.hatchability.com/Vaisala.pdf https://en.wikipedia.org/wiki/Vapour_pressure_of_water

In [ ]:
OHare_hourly_summer = OHare_hourly_summer.assign(saturation_vapor_pressure = lambda x: 10 * 0.61078*np.exp((17.27*x.temperature) / (x.temperature + 237.3)),
                           vapor_pressure = lambda x: x.saturation_vapor_pressure * x.relative_humidity/100, #units of hPa
                          absolute_humidity = lambda x: 100 * x.vapor_pressure * 2.16679 / (x.temperature + 273.15),
                            humidity_change = lambda x: x.absolute_humidity - x.absolute_humidity.shift(1)) #convert pressure from hPA to Pa and temperature from Celsius to Kelvin

In [ ]:
OHare_hourly_summer.assign(pressure_change = lambda x: x.pressure_change + np.random.normal(0,0.05, len(OHare_hourly_summer))).query('temp_change > 2.5').plot.scatter('humidity_change', 'pressure_change', s = 0.1)

##### We repeat the graph from above rounding the temperature change to the nearest half degree but finding the average humidity change for each temperature change value instead of the average pressure change and find a satisfying result in the temperature changes greater than 0: the average change in the humidity at these values does seem to go up faster than the average pressure at the same value. This makes sense: the air can hold a greater mass of water at higher temperatures, and in the summer there is an abundance of available excess water to evaporate at these temperatures. This would go a long way in explaining why the pressure rises on average with an increase in temperature in the summer.

In [ ]:
OHare_hourly_summer.assign(temp_change_custom = lambda x: (x.temp_change * 2).round(0)/2).groupby('temp_change_custom')['humidity_change'].mean().reset_index().plot.scatter('temp_change_custom', 'humidity_change')

##### Since we expect pressure to be proportional to temperature and inversely proportional to humidity, we would expect a positive linear relationship between temperature and the product of pressure and humidity, and that is indeed what we get. We can see that the product of humidity and pressure cannot exceed the exponential curve at 100% relative humidity for a given temperature.

In [ ]:
fig, ax = plt.subplots()
OHare_hourly_summer.assign(hum_press = lambda x: x.station_level_pressure * x.absolute_humidity, temp_jit = lambda x: x.temperature\
                           + np.random.normal(0,0.2, len(OHare_hourly_summer))).plot.scatter('temp_jit', 'hum_press', c = "blue", alpha = 0.5, s = 0.1, ax = ax)
OHare_hourly_summer.assign(hum_press = lambda x: x.station_level_pressure * x.absolute_humidity, temp_jit = lambda x: x.temperature\
                           + np.random.normal(0,0.2, len(OHare_hourly_summer))).query('relative_humidity == 100').plot\
.scatter('temp_jit', 'hum_press', alpha = 1, c = 'red', s = 0.1, ax = ax)
lgend = plt.legend(["All Measurements" , "Measurements with 100% Relative Humidity"], fontsize = 6, **{'labelcolor': ['blue', 'red']}, loc = "upper left")
for handle in lgend.legend_handles:
    handle.set_sizes([10.0])


In [ ]:
hum_press_quantiles = pd.DataFrame(np.nanquantile(OHare_hourly_summer.assign(hum_press = lambda x: x.station_level_pressure * x.absolute_humidity, 
                           temp_custom = lambda x: (round(x.temperature * 2))/2)[['temp_custom', 'hum_press']]\
.pivot(columns = 'temp_custom', values = 'hum_press'), np.arange(0.1,1.1,0.1), axis = 0))


In [ ]:
hum_press_quantiles.columns = OHare_hourly_summer.assign(hum_press = lambda x: x.station_level_pressure * x.absolute_humidity, 
                           temp_custom = lambda x: (round(x.temperature * 2))/2)[['temp_custom', 'hum_press']]\
.pivot(columns = 'temp_custom', values = 'hum_press').columns

In [ ]:
hum_press_quantiles = hum_press_quantiles.T.reset_index()

In [ ]:
hum_press_quantiles = hum_press_quantiles.drop(57) # row with all NA values

In [ ]:
a = mplPolygon(list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,9])) + list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,1]))[::-1], **{'fc':'red', 'alpha':0.1})
b = mplPolygon(list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,8])) + list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,2]))[::-1], **{'fc':'red', 'alpha':0.3})
c = mplPolygon(list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,7])) + list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,3]))[::-1], **{'fc':'red', 'alpha':0.5})
d = mplPolygon(list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,6])) + list(zip(hum_press_quantiles['temp_custom'], hum_press_quantiles.iloc[:,4]))[::-1], **{'fc':'red', 'alpha':0.7})

fig,ax = plt.subplots()
ax.set_xlim(0,40)
ax.set_ylim(0,25000)
ax.add_patch(a)
ax.add_patch(b)
ax.add_patch(c)
ax.add_patch(d)



OHare_hourly_summer.assign(hum_press = lambda x: x.station_level_pressure * x.absolute_humidity, 
                           temp_custom = lambda x: (round(x.temperature * 2))/2)[['temp_custom', 'hum_press']]\
.pivot(columns = 'temp_custom', values = 'hum_press').mean().reset_index().plot.scatter('temp_custom', 0, ax = ax, s = 0.5, c='black')
ax.set_ylabel("Humidity * Pressure")
ax.set_xlabel("Temperature")
plt.legend(['Inner 80% quantile', 'Inner 60% quantile', 'Inner 40% quantile', 'Inner 20% quantile', 'mean'])

##### Except for the bottlenecks in our graph with so few observations that the quantiles barely spread out, we see that the product of humidity and pressure does increase with temperature until about 22C, at which point it levels off significantly. This suggests that the absolute humidity (and not just the relative humidity) flattens at these temperatures to compensate for the increase in temperature.

##### Temperature vs average absolute humidity graph has much the same shape:

In [ ]:
fig, ax = plt.subplots()
OHare_hourly_summer.assign(temp_custom = lambda x: (round(x.temperature * 2))/2)[['temp_custom', 'absolute_humidity']]\
.pivot(columns = 'temp_custom', values = 'absolute_humidity').mean().reset_index().plot.scatter('temp_custom', 0, ax = ax)
ax.set_ylabel('Average Absolute Humidity')
ax.set_xlabel('Temperature')

##### This concludes the exploratory data analysis. We've numerically and visually explored a number of relationships between variables and derived insights about seasonal variations in weather patterns and the effect of proximity to Lake Michigan.  In the future, we could further this work by building statistical models to make predictions.   